<a href="https://colab.research.google.com/github/YashSharma79/mimic_a_star/blob/main/MimicAStar.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

1. Generate data for project 1
2. Generate data for project 2

Input data:
For every planned path,
  for every step that the agent walks
    KnowledgeGrid:Local info
    no. of open cell,no. of blocked cells
  StartPosition:Global info

Output data:
  For every planned path,
    for every step that the agent walks
      which neighour the agent moves (left/right/up/down)

Loss function:
 ( Predicted neighbour that nn moves to - neighbour that agent moved to)^2

 Both are 4D vectors
 when agent moves left it is [1, 0, 0, 0]
 when agent moves right it is [0, 1, 0, 0]
 when agent moves up it is [0, 1, 0, 0]
 when agent moves down it is [0, 0, 0, 1]

First layer#
101*101 nodes
values of the nodes = 
{
  0 if cell was blocked
  1 if cell was open
  2 if this was start cell
}

1 hidden layer, which will be initialised with node values

3rd and final layer:
4 nodes 
Only 1 of the 4 nodes will have value = 1 (the one where the agent moved to given the input grid)
Rest are all 0

Every iteration in the executePlannedFunction is one data point
Input to executePlannedFunction is a planned path
Every iteration here means movement (l/r/u/d)

If the original is moving in such a way that it frequently encounters blocked cell
Then the nn should do that as well

Initially, the grid is all 0s all empty
First data point will be
All nodes will have value 0
One of the nodes will have value 2
If agent moved right
2nd node of Output layer will have value 1

Given 1000 examples of the above,

The NN would know that in this case, the agent chooses the neighbour to the right

How to use the NN to move our agent?

Once its trained,
you give it the same kind of input
Which means give it a newly generated grid, that was not part of the training data
If the output given is [1, 0, 0, 0]
Make the agent move left




knowledge.world.flatten
Converts the matrix into array that we pass to the first layer of nn

In [ ]:
import tensorflow as tf
import collections
from queue import PriorityQueue
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from math import sqrt
from ast import literal_eval
from sklearn.model_selection import train_test_split
from timeit import default_timer as timer

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
GridData = pd.read_excel('/content/drive/My Drive/Intro to AI/AIFinalProject.xlsx')

# Agent 1 -  OG Agent from Project 1

In [ ]:
MovementDict = {'Grid':[], 'Movement':[]}

In [ ]:
def Voyage(n,p):
  class Cell:
    # x = x-coordinate, y = y-coordinate in grid. Values from 0 to n-1
    def __init__(self, x, y,cost_from_start, total_cost, parent):
      self.x = x
      self.y = y
      self.cost_from_start = cost_from_start
      self.total_cost = total_cost
      self.parent = parent

    def __lt__(self, other):    # Added to add an object to the priority queue
        return self.total_cost < other.total_cost

  class Grid:
    def __init__(self, world, start, goal, path):
      self.iterations = 0
      self.world = world
      self.start = start
      self.goal = goal
      self.path = []
      self.restartValue = 0
      self.allCells = {}    # a dictionary to lookup a cell. Used when looking up its gScore

  def gridworld2(n,p):
    start = [0,0]; end = [n-1,n-1]
    gridworld = []

    for width in range(n):    # Loop creates n lists of n elements
      width = []
      for length in range(n):
        rand = np.random.random()
        if rand > p:
          width.append(0)
        else:
          width.append(1)
      gridworld.append(width)

    gridworld = np.array(gridworld)    # List of lists is turned into an array
    gridworld[start] = 0; gridworld[end] = 0
    return gridworld
  
  gridObj = Grid(gridworld2(n,p), Cell(0, 0, 0, (n*2)-2, (0,0)), Cell(n-1,n-1,0,0,(0,0)), [])
  knowledgeGrid = Grid(gridworld2(n,0), Cell(0, 0, 0, (n*2)-2, (0,0)), Cell(n-1,n-1,0,0,(0,0)), [])

  def isCellWithinGrid(x, y, n):
    if (x < 0) or (x > (n - 1)) or (y < 0) or (y > (n - 1)):
      return False

  def isCellOpen(cell, n, grid):
    if isCellWithinGrid(cell.x, cell.y, n) == False: # check if within gridworld bounds
      return False
    
    if grid[cell.x][cell.y] == 1:    # check if blocked
      return False
    
    return True

  def calculatefn(cell):
    goal = (n - 1, n - 1)
    estimate = calculateManhattanDistance(cell, goal)
    return estimate + cell.cost_from_start

  def calculateManhattanDistance(cell, goal):
    return abs(cell.x - goal[0]) + abs(cell.y - goal[1])

  # this is the planning phase
  def AStar(currentGrid, start):
    queue = PriorityQueue()
    gridObj.allCells = {}
    gridObj.allCells[(start.x, start.y)] = start
    queue.put((start.total_cost,start))    # first value in the tuple is priority(cost), second is the cell object
    goal = gridObj.goal

    while not queue.empty():   #while queue is not empty
      getter = queue.get()
      current = getter[1]

      if current.x == goal.x and current.y == goal.y:
        path = reconstructPath(current, start)
        return executePlannedPath(path) 
     
      rightCell = Cell(current.x + 1, current.y, current.cost_from_start + 1, 0, (current.x, current.y))     # right neighbour
      downCell = Cell(current.x, current.y + 1, current.cost_from_start + 1, 0, (current.x, current.y))    # down neighbour
      leftCell = Cell(current.x - 1, current.y, current.cost_from_start + 1, 0, (current.x, current.y))  # left neighbour
      topCell = Cell(current.x, current.y - 1, current.cost_from_start + 1, 0, (current.x, current.y))  # top neighbour
     
      neighbors = [rightCell, downCell, leftCell, topCell] # calculate the f(n) for each neighbour of current

      for i in neighbors:
        i.total_cost = calculatefn(i)         # calculate the f(n) for each neighbour of current
        isItOpen = isCellOpen(i, n, currentGrid)
        if isCellOpen(i,n, currentGrid):
          element = gridObj.allCells.get((i.x, i.y))
          if element is not None:
            ''' 
            If new path to neigbour costs less than previous path to it, then update cell.
            This cell has already been visited before, but we are just checking
            if the current path is less costlier than the one visited before
            '''
            if i.total_cost < gridObj.allCells[(i.x, i.y)].total_cost:
             gridObj.allCells[(i.x, i.y)] = i

          else:
           gridObj.allCells[(i.x, i.y)] = i      # this cell is being added first time
           queue.put((i.total_cost, i))
    
    # queue is empty but goal node has not been reached
    return False


  def reconstructPath(current, start):
    path = collections.deque()
    
    while True:
      if (current.x, current.y) == (start.x, start.y):
        break
      path.appendleft(current)
      current = gridObj.allCells[(current.parent[0], current.parent[1])]
    
    return path
  def directionArray(cell):
    if (cell.x-cell.parent[0], cell.y-cell.parent[1]) == (0,1):
      return [1,0,0,0] # Right
    if (cell.x-cell.parent[0], cell.y-cell.parent[1]) == (0,-1):
      return [0,1,0,0] # Left
    if (cell.x-cell.parent[0], cell.y-cell.parent[1]) == (-1,0):
      return [0,0,1,0] # Up
    if (cell.x-cell.parent[0], cell.y-cell.parent[1]) == (1,0):
      return [0,0,0,1] # Down

  def executePlannedPath(path):
    grid = gridObj.world
    goal = gridObj.goal
    isPathFeasible = True

    for index, current in enumerate(path):
      #print("Visiting ", current)
      gridObj.iterations += 1

      #print(current.x-current.parent[0], current.y-current.parent[1])
      og = knowledgeGrid.world[current.parent[0],current.parent[1]]
      og2 = knowledgeGrid.world[knowledgeGrid.goal.x,knowledgeGrid.goal.y]
      knowledgeGrid.world[current.parent[0],current.parent[1]]=2
      knowledgeGrid.world[knowledgeGrid.goal.x,knowledgeGrid.goal.y] = 3
      #print(knowledgeGrid.world)

      MovementDict['Movement'].append(directionArray(current))
      MovementDict['Grid'].append(knowledgeGrid.world.flatten().tolist())

      knowledgeGrid.world[current.parent[0],current.parent[1]] = og
      knowledgeGrid.world[knowledgeGrid.goal.x,knowledgeGrid.goal.y] = og2
      if current.x == goal.x and current.y == goal.y:
        MovementDict['Movement'].append([0,0,0,1])
        og = knowledgeGrid.world[current.x,current.y]
        og2 = knowledgeGrid.world[knowledgeGrid.goal.x,knowledgeGrid.goal.y]
        knowledgeGrid.world[current.x,current.y]=2
        knowledgeGrid.world[knowledgeGrid.goal.x,knowledgeGrid.goal.y] = 3
        #print(knowledgeGrid.world)
        MovementDict['Grid'].append(knowledgeGrid.world.flatten().tolist())
        knowledgeGrid.world[current.x,current.y] = og
        knowledgeGrid.world[knowledgeGrid.goal.x,knowledgeGrid.goal.y] = og2
        return gridObj.path

      if grid[current.x][current.y] == 1:
        knowledgeGrid.world[current.x][current.y] = 1
        isPathFeasible = False

        # this should be the restart point now
        setRestartPoint(gridObj.allCells[current.parent[0], current.parent[1]], 0)
        return None

      else:
        gridObj.path.append(current)

      # update the neighbours in the actual grid   
      # update the blockedNeighboursCount   
      if isCellWithinGrid(current.x + 1, current.y, n) and grid[current.x + 1][current.y] == 1:
        knowledgeGrid[current.x + 1, current.y] = 1
      
      if isCellWithinGrid(current.x + 1, current.y, n) and grid[current.x + 1][current.y] == 1:
        knowledgeGrid[current.x][current.y + 1] = 1
      
      if isCellWithinGrid(current.x - 1, current.y, n) and grid[current.x - 1][current.y] == 1:
        knowledgeGrid[current.x + 1][current.y] = 1
      
      if isCellWithinGrid(current.x - 1, current.y, n) and grid[current.x][current.y - 1] == 1:
        knowledgeGrid[current.x + 1][current.y] = 1
  
  def setRestartPoint(restartPoint, restartValue):
    for i in range(0, restartValue, 1):
      restartPoint = gridObj.allCells[(restartPoint.parent[0], restartPoint.parent[1])]

    gridObj.start = restartPoint
    return

  def restartController(current):
    # in a loop run the restart again and again
    # starting from the current cell's parent to further up
    while (current.x != 0 and current.y != 0):
      current = gridObj.allCells[(current.parent[0], current.parent[1])]
      gridObj.start = current

  def repeatedAStar():
    generateSolvableGrid()
    while True:
        if len(gridObj.path) > 0:
          previousStartPoint = gridObj.path[len(gridObj.path)-1]

          # if we are restarting from the same point where we restarted previously, then dont add this to the path again
          if gridObj.start.x != previousStartPoint.x and gridObj.start.y != previousStartPoint.y:
            gridObj.path.append(gridObj.start)

        path = AStar(knowledgeGrid.world, gridObj.start)
        if path is None:
          continue

        elif path == False:
          return False

        # final path to goal found
        else:
          gridObj.world[0][0] = 5
          gridObj.world[gridObj.start.x][gridObj.start.y] = 5
          
          for i in gridObj.path:
            gridObj.world[i.x][i.y] = 5

          gridObj.world[gridObj.goal.x][gridObj.goal.y] = 5
          print(gridObj.world)
          print("Total iterations = ", gridObj.iterations)
          return True

  # this is the planning phase
  def AStar(currentGrid, start):
    queue = PriorityQueue()
    gridObj.allCells = {}
    gridObj.allCells[(start.x, start.y)] = start
    queue.put((start.total_cost,start))    # first value in the tuple is priority(cost), second is the cell object
    goal = gridObj.goal

    while not queue.empty():   #while queue is not empty
      getter = queue.get()
      current = getter[1]

      if current.x == goal.x and current.y == goal.y:
        path = reconstructPath(current, start)
        return executePlannedPath(path) 
     
      rightCell = Cell(current.x + 1, current.y, current.cost_from_start + 1, 0, (current.x, current.y))     # right neighbour
      downCell = Cell(current.x, current.y + 1, current.cost_from_start + 1, 0, (current.x, current.y))    # down neighbour
      leftCell = Cell(current.x - 1, current.y, current.cost_from_start + 1, 0, (current.x, current.y))  # left neighbour
      topCell = Cell(current.x, current.y - 1, current.cost_from_start + 1, 0, (current.x, current.y))  # top neighbour
     
      neighbors = [rightCell, downCell, leftCell, topCell] # calculate the f(n) for each neighbour of current

      for i in neighbors:
        i.total_cost = calculatefn(i)         # calculate the f(n) for each neighbour of current
        isItOpen = isCellOpen(i, n, currentGrid)
        if isCellOpen(i,n, currentGrid):
          element = gridObj.allCells.get((i.x, i.y))
          if element is not None:
            ''' 
            If new path to neigbour costs less than previous path to it, then update cell.
            This cell has already been visited before, but we are just checking
            if the current path is less costlier than the one visited before
            '''
            if i.total_cost < gridObj.allCells[(i.x, i.y)].total_cost:
             gridObj.allCells[(i.x, i.y)] = i

          else:
           gridObj.allCells[(i.x, i.y)] = i      # this cell is being added first time
           queue.put((i.total_cost, i))
    
    # queue is empty but goal node has not been reached
    return False

  def IsGridSolvable(currentGrid, start):
    queue = PriorityQueue()
    closedMap = {}
    closedMap[(start.x, start.y)] = start
    start.parent = (start.x, start.y)
    queue.put((start.total_cost,start))    # first value in the tuple is priority(cost), second is the cell object

    goal = knowledgeGrid.goal

    while not queue.empty():   #while queue is not empty
      getter = queue.get()
      current = getter[1]
    
      rightCell = Cell(current.x + 1, current.y, current.cost_from_start + 1, 0, (current.x, current.y))     # right neighbour
      downCell = Cell(current.x, current.y + 1, current.cost_from_start + 1, 0, (current.x, current.y))    # down neighbour
      leftCell = Cell(current.x - 1, current.y, current.cost_from_start + 1, 0, (current.x, current.y))  # left neighbour
      topCell = Cell(current.x, current.y - 1, current.cost_from_start + 1, 0, (current.x, current.y))  # top neighbour
     
      neighbors = [rightCell, downCell, leftCell, topCell] # calculate the f(n) for each neighbour of current

      for i in neighbors:
        i.total_cost = calculatefn(i)         # calculate the f(n) for each neighbour of current
        isItOpen = isCellOpen(i, n, currentGrid)
        if isCellOpen(i,n, currentGrid):
          element = gridObj.allCells.get((i.x, i.y))
          if element is not None:
            ''' 
            If new path to neigbour costs less than previous path to it, then update cell.
            This cell has already been visited before, but we are just checking
            if the current path is less costlier than the one visited before
            '''
            if i.total_cost < gridObj.allCells[(i.x, i.y)].total_cost:
             gridObj.allCells[(i.x, i.y)] = i

          else:
           gridObj.allCells[(i.x, i.y)] = i      # this cell is being added first time
           queue.put((i.total_cost, i))

      if current.x == goal.x and current.y == goal.y: # To Do: Check if it's the goal and then update the goal if not
            return True
    return False

  def generateSolvableGrid():
    is_grid_solvable = False
    while not is_grid_solvable:
      gridObj = Grid(gridworld2(n,p), Cell(0, 0, 0, (n*2)-2, (0,0)), Cell(n-1,n-1,0,0,(0,0)), [])
      #check if grid is solvable
      is_grid_solvable = IsGridSolvable(gridObj.world,  Cell(0, 0, 0, (n*2)-2, (0,0)))
      
    knowledgeGrid = Grid(gridworld2(n,0), Cell(0, 0, 0, (n*2)-2, (0,0)), Cell(n-1,n-1,0,0,(0,0)), [])
    
  def restartAtDifferentPoints():
    for i in range(0, (n*n - 1)):
      repeatedAStar()
    return

  return repeatedAStar()

for i in range(50): # Takes about 2 Min
  Voyage(101, 0.3)

In [ ]:
MovementFrame = pd.DataFrame(MovementDict)
print(MovementFrame)
Grids = np.asarray(MovementDict['Grid'])
Moves = np.asarray(MovementDict['Movement'])
(x_train, y_train ) = (Grids, Moves)
print('Number of grids =', len(MovementDict['Grid']))
print('x_train')
print('size of x_train', x_train.shape)
print(x_train)

print('Number of movements =', len(MovementDict['Movement']))
print('y_train')
print('size of y_train', y_train.shape)
print(y_train)

                                                    Grid      Movement
0      [2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...  [0, 0, 0, 1]
1      [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...  [0, 0, 0, 1]
2      [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...  [0, 0, 0, 1]
3      [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...  [1, 0, 0, 0]
4      [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...  [1, 0, 0, 0]
...                                                  ...           ...
21471  [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...  [1, 0, 0, 0]
21472  [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...  [1, 0, 0, 0]
21473  [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...  [1, 0, 0, 0]
21474  [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...  [1, 0, 0, 0]
21475  [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...  [0, 0, 0, 1]

[21476 rows x 2 columns]
Number of grids = 21476
x_train
size of x_train (21476, 10201)
[[2 0 0 ... 0 0 3]
 [0 0 0 ... 0 0 3]
 [0 0 0 ... 0 0 3]
 .

  [0, 0, 0, 0] in the output corresponds to reaching goal

# Agent 3 -  OG Agent from Project 2

In [ ]:
import collections
from queue import PriorityQueue
import numpy as np
import pandas as pd
#import matplotlib.pyplot as plt
from math import sqrt
from timeit import default_timer as timer
import random
import csv
#import xlsxwriter
Project2MovementDict = {'Visited':[],'Movement':[],'MinusInferance':[],'SensedNeighboursGrid':[]}

def PartialSensing(n, p):
    class Solver:
        def __init__(self):
            # a variable is a cell
            # constraints are all the equations that this cell is part of
            self.variables_to_constraints_map = {}

            # each constraint will a key value pair
            # key is a tuple, blocked_neighbours is a value
            # this is a list of constraints
            self.constraints = []

        def add_constraint(self, variable, n_list, value):
            variable_set = set()
            for i in n_list:
                #print('constraint')
                # print(i.x, i.y)
                variable_set.add((i.x * n + i.y))

            constraint = {}
            constraint[tuple(variable_set)] = value
            self.constraints.append(constraint)
            constraintIndex = len(self.constraints) - 1

            # adding constraint
            for v in variable_set:
                if self.variables_to_constraints_map.get(v) is None:
                    self.variables_to_constraints_map[v] = []
                self.variables_to_constraints_map[v].append(constraintIndex)

        def check_satisfaction_conditions(self, lhs, rhs):
            if rhs == 0:
                # all cells on lhs are free
                for i in range(0, len(lhs), 1):
                    cellX, cellY = divmod(lhs[i], n)
                    inferred_free(cellX, cellY)

            elif len(lhs) == rhs:
                # all cells on lhs are blocked
                for i in range(0, len(lhs), 1):
                    cellX, cellY = divmod(lhs[i], n)
                    inferred_blocked(cellX, cellY)

        def solve_constraint(self, cell, cell_value):
            # print("Solving constraint for ", cell.x, cell.y)
            # Solved when:
            # RHS becomes zero means all are free
            # Size of the list/set on left side = no. on the right side means all are blocked or all are free
            variable = cell.x * n + cell.y
            # get all constraint equations of this variable
            if self.variables_to_constraints_map.get(variable) is None:
                # print("No constraint exists for ", cell.x, cell.y)
                return

            constraints = self.variables_to_constraints_map[variable]
            for constraint_index in constraints:
                constraint = self.constraints[constraint_index]
                (lhs, rhs), = constraint.items()
                temp_lhs = list(lhs)
                temp_lhs.remove(variable)
                lhs = tuple(temp_lhs)

                if cell_value == 1:
                    rhs -= 1

                self.variables_to_constraints_map[variable].remove(constraint_index)
                constraint = {}
                constraint[lhs] = rhs
                self.constraints[constraint_index] = constraint
                self.check_satisfaction_conditions(lhs, rhs)
            return

    solver = Solver()

    def inferred_free(x, y):
        if knowledgeGrid.cellMap[(x, y)].confirmed:
            return
        knowledgeGrid.world[x, y] = 0
        knowledgeGrid.cellMap[(x, y)].confirmed = True
        # inform neighbours that this cell has been free
        informNeigboursAboutNewFreeCell(knowledgeGrid.cellMap[(x, y)])
        solver.solve_constraint(knowledgeGrid.cellMap[(x, y)], 0)

    def inferred_blocked(x, y):
        if knowledgeGrid.cellMap[(x, y)].confirmed:
            return

        knowledgeGrid.world[x, y] = 1
        knowledgeGrid.cellMap[(x, y)].confirmed = True
        # inform neighbours that this cell has been free
        informNeighboursAboutNewBlockedCell(knowledgeGrid.cellMap[(x, y)])
        solver.solve_constraint(knowledgeGrid.cellMap[(x, y)], 1)

    def agent_four_inference(cell, unknown_blocked):
        neighbours = getNeighbours(cell)
        for i in neighbours:
            if i.confirmed:
                if knowledgeGrid.world[i.x, i.y] == 1:
                    unknown_blocked -= 1
                # remove this from the equation because its value is known
                # if the value was 1, then that case was handled in the above line
                # if value was 0, then there is no effect on RHS
                neighbours.remove(i)

        solver.add_constraint(cell.x * n + cell.y, neighbours, unknown_blocked)
        return

    class Cell:
        # x = x-coordinate, y = y-coordinate in grid. Values from 0 to n-1
        def __init__(self, x, y, cost_from_start, total_cost, parent):
            self.x = x
            self.y = y
            self.cost_from_start = cost_from_start
            self.total_cost = total_cost
            self.total_neighbours = 0
            self.total_blocked_neighbours = 0
            self.known_neighbours = 0
            self.known_blocked_neighbours = 0
            self.parent = parent
            self.confirmed = False
            self.visited = False

        def __lt__(self, other):  # Added to add an object to the priority queue
            return self.total_cost < other.total_cost

    class Grid:
        def __init__(self, world, start, goal, path):
            self.world = world
            self.start = start
            self.goal = goal
            self.path = []
            self.cellMap = {}  # a dictionary to lookup a cell.

    def gridworld2(n, p):
        start = [0, 0];
        end = [n - 1, n - 1]
        gridworld = []

        for width in range(n):  # Loop creates n lists of n elements
            width = []
            for length in range(n):
                rand = np.random.random()
                if rand > p:
                    width.append(0)
                else:
                    width.append(1)
            gridworld.append(width)

        gridworld = np.array(gridworld)  # List of lists is turned into an array
        gridworld[start] = 0;
        gridworld[end] = 0
        return gridworld

    gridObj = Grid(gridworld2(n, p), Cell(0, 0, 0, (n * 2) - 2, (0, 0)), Cell(n - 1, n - 1, 0, 0, (0, 0)), [])
    knowledgeGrid = Grid(gridworld2(n, 0), Cell(0, 0, 0, (n * 2) - 2, (0, 0)), Cell(n - 1, n - 1, 0, 0, (0, 0)), [])
    KnowledgeGridMinusInferance = Grid(gridworld2(n, 0), Cell(0, 0, 0, (n * 2) - 2, (0, 0)), Cell(n - 1, n - 1, 0, 0, (0, 0)), []) #declaration
    InferanceGrid = Grid(gridworld2(n, 0), Cell(0, 0, 0, (n * 2) - 2, (0, 0)), Cell(n - 1, n - 1, 0, 0, (0, 0)), []) #declaration
    visited_grid = gridworld2(n, 0)

    
    def isCellWithinGrid(x, y, n):
        if (x < 0) or (x > (n - 1)) or (y < 0) or (y > (n - 1)):
            return False
        return True

    def isCellOpen(cell, n, grid):
        if isCellWithinGrid(cell.x, cell.y, n) == False:  # check if within gridworld bounds
            return False

        if grid.world[cell.x, cell.y] == 1:  # check if blocked
            return False

        return True

    def calculatefn(cell):
        goal = (n - 1, n - 1)
        estimate = calculateManhattanDistance(cell, goal)
        return estimate + cell.cost_from_start

    # Probability Start
    def calculateProbilityfn(cell):
        goal = (n - 1, n - 1)

        # when some neighbours are unknown
        if (cell.total_neighbours - cell.known_neighbours) != 0:
            estimateWithProbability = round(calculateManhattanDistance(cell, goal) * (((
                                                                                                   cell.total_blocked_neighbours - cell.known_blocked_neighbours) / (
                                                                                                   cell.total_neighbours - cell.known_neighbours)) + 1))

        # when all neighbours of a cell are known, no need to involve probabilistic calculations for them
        else:
            estimateWithProbability = calculateManhattanDistance(cell, goal)
        # print('Estimate : ' + str(estimateWithProbability))
        return estimateWithProbability + cell.cost_from_start
        # Probability End

    def calculateManhattanDistance(cell, goal):
        return abs(cell.x - goal[0]) + abs(cell.y - goal[1])

    def getNeighbours(cell):
        # east, south, west, north, ne, se, sw, nw in that order
        x = [1, 0, -1, 0, 1, 1, -1, -1]
        y = [0, 1, 0, -1, -1, 1, 1, -1]

        neighbours = []
        #the parent being added here is wrong
        #this thing is resetting the parent
        #
        for i in range(8):
            if isCellWithinGrid(cell.x + x[i], cell.y + y[i], n):
                if knowledgeGrid.cellMap.get((cell.x + x[i], cell.y + y[i])) is None:
                    knowledgeGrid.cellMap[(cell.x + x[i], cell.y + y[i])] = Cell(cell.x + x[i], cell.y + y[i], 0, 0,
                                                                                 (cell.x, cell.y))
                neighbours.append(knowledgeGrid.cellMap[(cell.x + x[i], cell.y + y[i])])

        return neighbours

    def senseNeighbours(cell):
        ECell = Cell(cell.x + 1, cell.y, cell.cost_from_start + 1, 0, (cell.x, cell.y))  # East neighbour
        SCell = Cell(cell.x, cell.y + 1, cell.cost_from_start + 1, 0, (cell.x, cell.y))  # South neighbour
        WCell = Cell(cell.x - 1, cell.y, cell.cost_from_start + 1, 0, (cell.x, cell.y))  # West neighbour
        NCell = Cell(cell.x, cell.y - 1, cell.cost_from_start + 1, 0, (cell.x, cell.y))  # North neighbour
        NECell = Cell(cell.x + 1, cell.y - 1, cell.cost_from_start + 1, 0, (cell.x, cell.y))  # North East neighbour
        SECell = Cell(cell.x + 1, cell.y + 1, cell.cost_from_start + 1, 0, (cell.x, cell.y))  # South East neighbour
        SWCell = Cell(cell.x - 1, cell.y + 1, cell.cost_from_start + 1, 0, (cell.x, cell.y))  # South West neighbour
        NWCell = Cell(cell.x - 1, cell.y - 1, cell.cost_from_start + 1, 0, (cell.x, cell.y))  # North West neighbour

        potential_neighbours = [ECell, SCell, WCell, NCell, NECell, SECell, SWCell, NWCell]
        neighbours = [i for i in potential_neighbours if isCellWithinGrid(i.x, i.y, n)]
        total = 0
        blocked = 0
        for i in neighbours:
            total += 1
            if gridObj.world[i.x, i.y] == 1:
                blocked += 1
        cell.total_neighbours = total
        cell.total_blocked_neighbours = blocked
        return cell

    def getKnownNeighbours(cell):
        # east, south, west, north, ne, se, sw, nw in that order
        x = [1, 0, -1, 0, 1, 1, -1, -1]
        y = [0, 1, 0, -1, -1, 1, 1, -1]

        neighbours = []
        for i in range(8):
            if isCellWithinGrid(cell.x + x[i], cell.y + y[i], n):
                neighbourCell = knowledgeGrid.cellMap.get((cell.x + x[i], cell.y + y[i]))
                if neighbourCell is not None and neighbourCell.confirmed:
                    cell.known_neighbours += 1
                    if knowledgeGrid.world[cell.x + x[i], cell.y + y[i]] == 1:
                        cell.known_blocked_neighbours += 1
        return cell

    def updateKnownNeighbours(cell):
        knowledgeGrid.cellMap[(cell.x, cell.y)].known_neighbours += 1
        inferenceEngine(cell)

    def updateKnownBlockedNeighbours(cell):
        cell.known_blocked_neighbours += 1
        cell.known_neighbours += 1
        knowledgeGrid.cellMap[(cell.x, cell.y)] = cell
        inferenceEngine(cell)

    def inferenceEngine(cell):
        if cell.total_blocked_neighbours == cell.known_blocked_neighbours:
            markRemainingNeighboursAsFree(cell)

        # if totalFreeNeighbours = knownFreeNeighbours
        if (cell.total_neighbours - cell.total_blocked_neighbours
                == (cell.known_neighbours - cell.known_blocked_neighbours)):
            markRemainingNeighboursAsBlocked(cell)

        # Early Exit Function
        if cell.total_blocked_neighbours == cell.total_neighbours - 1:
            markRemainingNeighboursAsBlocked(cell)

    def markRemainingNeighboursAsFree(cell):
        neighbours = getNeighbours(cell)
        for i in neighbours:
            # this new freely marked cell must inform its neighbours which have been visited
            if i.confirmed == False:
                knowledgeGrid.world[i.x, i.y] = 0
                cell.known_neighbours += 1
                knowledgeGrid.cellMap[(i.x, i.y)] = cell
                i.confirmed = True
                knowledgeGrid.cellMap[(i.x, i.y)] = i
                # inform the neighbours
                informNeigboursAboutNewFreeCell(i)
                solver.solve_constraint(knowledgeGrid.cellMap[(i.x, i.y)], 0)

    def markRemainingNeighboursAsBlocked(cell):
        neighbours = getNeighbours(cell)
        for i in neighbours:
            # this new blocked cell must inform its neighbours which have been visited
            if i.confirmed == False:
                # print("Marking ", i.x, i.y, " as a blocked neighbour of ", cell.x, cell.y)
                cell.known_blocked_neighbours += 1
                cell.known_neighbours += 1
                knowledgeGrid.cellMap[(i.x, i.y)] = cell
                knowledgeGrid.world[i.x, i.y] = 1
                i.confirmed = True
                knowledgeGrid.cellMap[(i.x, i.y)] = i
                informNeighboursAboutNewBlockedCell(i)
                solver.solve_constraint(knowledgeGrid.cellMap[(i.x, i.y)], 1)

    def informNeigboursAboutNewFreeCell(cell):
        neighbours = getNeighbours(cell)
        for i in neighbours:
            if i.visited:
                # print(cell.x, cell.y, " is a known free neighbour of ", i.x, i.y)
                updateKnownNeighbours(i)

    def informNeighboursAboutNewBlockedCell(cell):
        neighbours = getNeighbours(cell)
        for i in neighbours:
            if i.visited:
                # print(cell.x, cell.y, " is a known blocked neighbour of ", i.x, i.y)
                updateKnownBlockedNeighbours(i)
    #Ucode Start
    def IsGridSolvable(currentGrid, start):
        queue = PriorityQueue()
        closedMap = {}
        closedMap[(start.x, start.y)] = start
        start.parent = (start.x, start.y)
        queue.put((start.total_cost, start))  # first value in the tuple is priority(cost), second is the cell object

        goal = knowledgeGrid.goal

        while not queue.empty():  # while queue is not empty
            getter = queue.get()
            current = getter[1]

            rightCell = Cell(current.x + 1, current.y, current.cost_from_start + 1, 0,
                             (current.x, current.y))  # right neighbour
            downCell = Cell(current.x, current.y + 1, current.cost_from_start + 1, 0,
                            (current.x, current.y))  # down neighbour
            leftCell = Cell(current.x - 1, current.y, current.cost_from_start + 1, 0,
                            (current.x, current.y))  # left neighbour
            topCell = Cell(current.x, current.y - 1, current.cost_from_start + 1, 0,
                           (current.x, current.y))  # top neighbour

            neighbors = [rightCell, downCell, leftCell, topCell]  # calculate the f(n) for each neighbour of current

            for i in neighbors:
                i.total_cost = calculatefn(i)  # calculate the f(n) for each neighbour of current
                isItOpen = isCellOpen(i, n, currentGrid)
                if isCellOpen(i, n, currentGrid):
                    element = gridObj.cellMap.get((i.x, i.y))
                    if element is not None:
                        ''' 
                        If new path to neigbour costs less than previous path to it, then update cell.
                        This cell has already been visited before, but we are just checking
                        if the current path is less costlier than the one visited before
                        '''
                        if i.total_cost < gridObj.cellMap[(i.x, i.y)].total_cost:
                            gridObj.cellMap[(i.x, i.y)] = i

                    else:
                        gridObj.cellMap[(i.x, i.y)] = i  # this cell is being added first time
                        queue.put((i.total_cost, i))

            if current.x == goal.x and current.y == goal.y:  # To Do: Check if it's the goal and then update the goal if not
                return True
        return False

    def generateSolvableGrid():
        is_grid_solvable = False
        while not is_grid_solvable:
            gridObj = Grid(gridworld2(n, p), Cell(0, 0, 0, (n * 2) - 2, (0, 0)), Cell(n - 1, n - 1, 0, 0, (0, 0)), [])
            # check if grid is solvable
            is_grid_solvable = IsGridSolvable(gridObj, Cell(0, 0, 0, (n * 2) - 2, (0, 0)))

        knowledgeGrid = Grid(gridworld2(n, 0), Cell(0, 0, 0, (n * 2) - 2, (0, 0)), Cell(n - 1, n - 1, 0, 0, (0, 0)), [])
    #Ucode end
    # this is the planning phase
    def AStar(knowledgeGrid, start):
        # print("Starting from ", start.x, start.y)
        queue = PriorityQueue()
        closedMap = {}
        closedMap[(start.x, start.y)] = start
        queue.put((start.total_cost, start))  # first value in the tuple is priority(cost), second is the cell object
        goal = knowledgeGrid.goal

        while not queue.empty():  # while queue is not empty
            getter = queue.get()
            current = getter[1]
            if isCellOpen(current, n, knowledgeGrid) is False and isCellWithinGrid(current.x, current.y, n) is True:
                knowledgeGrid.world[current.x, current.y] = 1

            if current.x == goal.x and current.y == goal.y:
                path = reconstructPath(current, start, closedMap)
                return executePlannedPath(path)

            rightCell = Cell(current.x + 1, current.y, current.cost_from_start + 1, 0,
                             (current.x, current.y))  # right neighbour
            downCell = Cell(current.x, current.y + 1, current.cost_from_start + 1, 0,
                            (current.x, current.y))  # down neighbour
            leftCell = Cell(current.x - 1, current.y, current.cost_from_start + 1, 0,
                            (current.x, current.y))  # left neighbour
            topCell = Cell(current.x, current.y - 1, current.cost_from_start + 1, 0,
                           (current.x, current.y))  # top neighbour

            neighbors = [rightCell, downCell, leftCell, topCell]  # calculate the f(n) for each neighbour of current

            for i in neighbors:
                # i.total_cost = calculatefn(i)  # calculate the f(n) for each neighbour of current

                if knowledgeGrid.cellMap.get((i.x, i.y)) and knowledgeGrid.cellMap[(i.x, i.y)].visited:
                    i.total_cost = calculateProbilityfn(knowledgeGrid.cellMap[(i.x, i.y)])
                else:
                    i.total_cost = calculatefn(i)
                    #knowledgeGrid.cellMap[(i.x, i.y)].parent = (current.x, current.y)

                if isCellOpen(i, n, knowledgeGrid):
                    element = closedMap.get((i.x, i.y))
                    if element is not None:
                        ''' 
                        If new path to neigbour costs less than previous path to it, then update cell.
                        This cell has already been visited before, but we are just checking
                        if the current path is less costlier than the one visited before
                        '''
                        if i.total_cost < closedMap[(i.x, i.y)].total_cost:
                            closedMap[(i.x, i.y)] = i

                    else:
                        closedMap[(i.x, i.y)] = i  # this cell is being added first time
                        queue.put((i.total_cost, i))
            previous = current
        # queue is empty but goal node has not been reached
        return False

    def reconstructPath(current, start, closedMap):
        path = collections.deque()

        while True:
            if (current.x, current.y) == (start.x, start.y):
                break
            path.appendleft(current)
            current = closedMap[(current.parent[0], current.parent[1])]

        return path

    def directionArray(cell):
        if (cell.x - cell.parent[0], cell.y - cell.parent[1]) == (0, 1):
            return [1, 0, 0, 0]  # Right
        if (cell.x - cell.parent[0], cell.y - cell.parent[1]) == (0, -1):
            return [0, 1, 0, 0]  # Left
        if (cell.x - cell.parent[0], cell.y - cell.parent[1]) == (-1, 0):
            return [0, 0, 1, 0]  # Up
        if (cell.x - cell.parent[0], cell.y - cell.parent[1]) == (1, 0):
            return [0, 0, 0, 1]  # Down
        
        else:
            print("Parent of", cell.x, cell.y, " is", cell.parent[0], cell.parent[1])
            print("Might return none for", cell.x - cell.parent[0], cell.y - cell.parent[1])

    def executePlannedPath(path):
        grid = gridObj.world
        goal = gridObj.goal

        # print('total blocked at 00   ' + str(gridObj.cellMap[0, 0].total_blocked_neighbours))
        InferanceGrid.world[0, 0] = gridObj.cellMap[0, 0].total_blocked_neighbours

        prev = (knowledgeGrid.start.x, knowledgeGrid.start.y)
        for index, current in enumerate(path):

            KnowledgeGridMinusInferance.world[current.x, current.y] = 2


            alreadyConfirmed = False
            if knowledgeGrid.cellMap.get((current.x, current.y)) is None:
                knowledgeGrid.cellMap[(current.x, current.y)] = current

            else:
                alreadyConfirmed = True
                current = knowledgeGrid.cellMap[(current.x, current.y)]

            current.parent = prev


            if directionArray(current) == None:
                print("None array for", current.x, current.y)


            if current.x == goal.x and current.y == goal.y:
                KnowledgeGridMinusInferance.world[current.x, current.y] = 3
                
                Project2MovementDict['Visited'].append(visited_grid.flatten().tolist())
                Project2MovementDict['Movement'].append(directionArray(current))
                Project2MovementDict['MinusInferance'].append(KnowledgeGridMinusInferance.world.flatten().tolist())
                Project2MovementDict['SensedNeighboursGrid'].append(InferanceGrid.world.flatten().tolist())
            
                return knowledgeGrid.path
            #Ucode Start
            #Project2MovementDict['Grid'].append(knowledgeGrid.world.flatten().tolist())
  
            Project2MovementDict['Visited'].append(visited_grid.flatten().tolist())
            Project2MovementDict['Movement'].append(directionArray(current))
            Project2MovementDict['MinusInferance'].append(KnowledgeGridMinusInferance.world.flatten().tolist())
            Project2MovementDict['SensedNeighboursGrid'].append(InferanceGrid.world.flatten().tolist())
            
            #Ucode End
            # hit blocked cell
            if grid[current.x, current.y] == 1:
                current.confirmed = True
                # print("Blocked by ", current.x, current.y)
                knowledgeGrid.world[current.x, current.y] = 1

                KnowledgeGridMinusInferance.world[current.x, current.y] = 1

                # setting restart point
                knowledgeGrid.start = gridObj.cellMap[current.parent[0], current.parent[1]]

                KnowledgeGridMinusInferance.world[current.parent[0], current.parent[1]]= 2

                # adding to the cell map
                knowledgeGrid.cellMap[(current.x, current.y)] = current

                # inform the neighbours that this is blocked
                if alreadyConfirmed == False:
                    informNeighboursAboutNewBlockedCell(current)

                solver.solve_constraint(current, 1)

                return None

            else:
                # can only sense neighbours of free cells
                visited_grid[current.x, current.y] += 1
                current.confirmed = True
                current.visited = True
                current.total_neighbours = gridObj.cellMap[(current.x, current.y)].total_neighbours
                current.total_blocked_neighbours = gridObj.cellMap[(current.x, current.y)].total_blocked_neighbours
                InferanceGrid.world[(current.x, current.y)] = gridObj.cellMap[(current.x, current.y)].total_blocked_neighbours
                if alreadyConfirmed == False:
                    current = getKnownNeighbours(current)

                knowledgeGrid.cellMap[(current.x, current.y)] = current
                knowledgeGrid.path.append(current)
                inferenceEngine(current)

                # inform the neighbours that this is free
                if alreadyConfirmed == False:
                    informNeigboursAboutNewFreeCell(current)

                # advanced inference
                agent_four_inference(current, current.total_blocked_neighbours)
                solver.solve_constraint(current, 0)
                prev = (current.x, current.y)


    def countBlockedNeighboursInRealGrid():
        # populate the information about blocked neighbours in the real grid
        for i in range(n):
            for j in range(n):
                cell = Cell(i, j, 0, 0, (0, 0))
                cell = senseNeighbours(cell)

                # create object for all cells in the grid
                gridObj.cellMap[(cell.x, cell.y)] = cell

    def repeatedAStar():
        generateSolvableGrid()

        #printGridInItsFullGlory(visited_grid)
        countBlockedNeighboursInRealGrid()

        # mark start as confirmed
        knowledgeGrid.start.confirmed = True
        knowledgeGrid.start.total_blocked_neighbours = gridObj.cellMap[(0, 0)].total_blocked_neighbours
        knowledgeGrid.cellMap[(0, 0)] = knowledgeGrid.start

        # print(gridObj.world)
        # print()
        agent_four_inference(knowledgeGrid.start, knowledgeGrid.start.total_blocked_neighbours)

        while True:
            if len(knowledgeGrid.path) > 0:
                previousStartPoint = knowledgeGrid.path[len(knowledgeGrid.path) - 1]

                # if we are restarting from the same point where we restarted previously, then dont add this to the path again
                if knowledgeGrid.start.x != previousStartPoint.x and knowledgeGrid.start.y != previousStartPoint.y:
                    knowledgeGrid.path.append(knowledgeGrid.start)

            path = AStar(knowledgeGrid, knowledgeGrid.start)
            if path is None:
                continue

            elif path == False:
                # print("Knowledge grid line 279:")
                # print(knowledgeGrid.world)
                return False

            else:
                knowledgeGrid.world[0, 0] = 5
                knowledgeGrid.world[knowledgeGrid.start.x, knowledgeGrid.start.y] = 5

                for i in knowledgeGrid.path:
                    # print('path')
                    # print(i.x, i.y)
                    knowledgeGrid.world[i.x, i.y] = 5

                knowledgeGrid.world[knowledgeGrid.goal.x, knowledgeGrid.goal.y] = 5
                # print("Knowledge grid line 291:")
                #print(knowledgeGrid.world)
                #printGridInItsFullGlory(visited_grid) 
                return True
    return repeatedAStar()


for i in range(2):
    PartialSensing(101, 0.3)
    print(i)

#import pandas as p
   # df = pd.DataFrame(MovementDict)
#df = pd.DataFrame.from_dict(MovementDict, orient='index')
#df = df.transpose()
#df.to_excel('Project 2 data file 1.xlsx', index=False)
#print('completed')

# Neural network to mimic Agent 1

1. Find out how to look inside the history object

In [ ]:
#80-20 split for train and test

#x_train, x_test, y_train, y_test = train_test_split(x_train, y_train, test_size=0.2, random_state =1)

train_in = np.reshape( x_train, (-1, 101, 101) )
train_out = np.reshape( y_train, (-1, 4) )

#define neural network layers, size of each layer (no. of nodes in each layer)
grid_input = tf.keras.layers.Input( shape = (101,101) )
flatten_image = tf.keras.layers.Flatten()( grid_input )
#dense_1 is the first hidden layer (fully connected)
dense_1 = tf.keras.layers.Dense( units = 1500, activation = tf.nn.relu )( flatten_image )

#dense_2 is the second hidden layer
dense_2 = tf.keras.layers.Dense( units = 1000, activation = tf.nn.relu )( dense_1 )

#dense_3 is the third hidden layer
dense_3 = tf.keras.layers.Dense( units = 1000, activation = tf.nn.relu )( dense_2 )

#output layer
movement_output = tf.keras.layers.Dense( units = 4, activation = None )( dense_3 )

probabilities = tf.keras.layers.Softmax()( movement_output )

model = tf.keras.Model( inputs = grid_input, outputs = probabilities )
model.compile( optimizer = 'adam', loss = 'categorical_crossentropy', metrics = ['accuracy'] )

model.fit( train_in, train_out, epochs = 30, verbose = 0)
#print(history)
model.predict(train_in[0:1])

array([[4.9825934e-09, 1.1096784e-12, 6.4220584e-20, 1.0000000e+00]],
      dtype=float32)

In [ ]:
model.fit( train_in, train_out, epochs = 20)

Epoch 1/20
414/414 [==============================] - 5s 13ms/step - loss: 0.4408 - accuracy: 0.7676
Epoch 2/20
414/414 [==============================] - 5s 12ms/step - loss: 0.4369 - accuracy: 0.7639
Epoch 3/20
414/414 [==============================] - 5s 13ms/step - loss: 0.4307 - accuracy: 0.7767
Epoch 4/20
414/414 [==============================] - 5s 13ms/step - loss: 0.4261 - accuracy: 0.7751
Epoch 5/20
414/414 [==============================] - 5s 13ms/step - loss: 0.4222 - accuracy: 0.7809
Epoch 6/20
414/414 [==============================] - 5s 13ms/step - loss: 0.4169 - accuracy: 0.7877
Epoch 7/20
414/414 [==============================] - 5s 13ms/step - loss: 0.4147 - accuracy: 0.7851
Epoch 8/20
414/414 [==============================] - 5s 13ms/step - loss: 0.4114 - accuracy: 0.7833
Epoch 9/20
414/414 [==============================] - 5s 13ms/step - loss: 0.4101 - accuracy: 0.7830
Epoch 10/20
414/414 [==============================] - 5s 13ms/step - loss: 0.4047 - accura

# Movement by neural network

In [ ]:
def NeuralVoyage(n,p):
  class Cell:
    # x = x-coordinate, y = y-coordinate in grid. Values from 0 to n-1
    def __init__(self, x, y,cost_from_start, total_cost, parent):
      self.x = x
      self.y = y
      self.cost_from_start = cost_from_start
      self.total_cost = total_cost
      self.parent = parent

    def __lt__(self, other):    # Added to add an object to the priority queue
        return self.total_cost < other.total_cost

  class Grid:
    def __init__(self, world, start, goal, path):
      self.iterations = 0
      self.world = world
      self.start = start
      self.goal = goal
      self.path = []
      self.restartValue = 0
      self.allCells = {}    # a dictionary to lookup a cell. Used when looking up its gScore

  def gridworld2(n,p):
    start = [0,0]; end = [n-1,n-1]
    gridworld = []

    for width in range(n):    # Loop creates n lists of n elements
      width = []
      for length in range(n):
        rand = np.random.random()
        if rand > p:
          width.append(0)
        else:
          width.append(1)
      gridworld.append(width)

    gridworld = np.array(gridworld)    # List of lists is turned into an array
    gridworld[start] = 0; gridworld[end] = 0
    return gridworld
  
  gridObj = Grid(gridworld2(n,p), Cell(0, 0, 0, (n*2)-2, (0,0)), Cell(n-1,n-1,0,0,(0,0)), [])
  knowledgeGrid = Grid(gridworld2(n,0), Cell(0, 0, 0, (n*2)-2, (0,0)), Cell(n-1,n-1,0,0,(0,0)), [])

  def isCellWithinGrid(x, y, n):
    if (x < 0) or (x > (n - 1)) or (y < 0) or (y > (n - 1)):
      return False

  def isCellOpen(cell, n, grid):
    if isCellWithinGrid(cell.x, cell.y, n) == False: # check if within gridworld bounds
      return False
    
    if grid[cell.x][cell.y] == 1:     # check if blocked
      return False
    
    return True

  def calculatefn(cell):
    goal = (n - 1, n - 1)
    estimate = calculateManhattanDistance(cell, goal)
    return estimate + cell.cost_from_start

  def calculateManhattanDistance(cell, goal):
    return abs(cell.x - goal[0]) + abs(cell.y - goal[1])

  def Direct(currentGrid, current):
      world = currentGrid
      world2 = world
      world[current.x,current.y]=2
      world[knowledgeGrid.goal.x,knowledgeGrid.goal.y]=3
      #print(world)
      world = world.flatten()
      test_in = np.reshape(world, (-1, 101, 101) )
      predicted_move = model.predict(test_in)
      #print(predicted_move)
      predicted_move = predicted_move[0].tolist()
      direction = predicted_move.index(np.max(predicted_move))
      fourth = predicted_move.index(np.min(predicted_move))
      predicted_move[direction] = 0
      second = predicted_move.index(np.max(predicted_move))
      predicted_move[second] = 0
      third = predicted_move.index(np.max(predicted_move))
      predicted_move[third] = 0
      world = world2
      world[current.x,current.y]=0
      #print(direction, second, third, fourth)
      return (direction, second, third, fourth)
      
  # this is the planning phase  
  def NN(currentGrid, start):
    gridObj.allCells = {}
    gridObj.allCells[(start.x, start.y)] = start
    print('Starting at: ', (start.x, start.y))
    current = start
    prev = (current.x, current.y)
    goal = gridObj.goal
    while (current.x, current.y) != (goal.x, goal.y):
      print((current.x, current.y))
      rightCell = Cell(current.x + 1, current.y, current.cost_from_start + 1, 0,
                      (current.x, current.y))  # right neighbour
      downCell = Cell(current.x, current.y + 1, current.cost_from_start + 1, 0,
                      (current.x, current.y))  # down neighbour
      leftCell = Cell(current.x - 1, current.y, current.cost_from_start + 1, 0,
                      (current.x, current.y))  # left neighbour
      topCell = Cell(current.x, current.y - 1, current.cost_from_start + 1, 0,
                      (current.x, current.y))  # top neighbour

      neighbors = [rightCell, leftCell, topCell, downCell]  # calculate the f(n) for each neighbour of current
      move = Direct(knowledgeGrid.world, current)
      get1 = gridObj.allCells.get((neighbors[move[0]].x, neighbors[move[0]].y))
      get2 = gridObj.allCells.get((neighbors[move[1]].x, neighbors[move[1]].y))
      get3 = gridObj.allCells.get((neighbors[move[2]].x, neighbors[move[2]].y))
      get4 = gridObj.allCells.get((neighbors[move[3]].x, neighbors[move[3]].y))
      if isCellOpen(neighbors[move[0]], n, currentGrid) and get1 is None:
        gridObj.allCells[(neighbors[move[0]].x, neighbors[move[0]].y)] = neighbors[move[0]]
        current = neighbors[move[0]]
      elif isCellOpen(neighbors[move[1]], n, currentGrid) and get2 is None:
        gridObj.allCells[(neighbors[move[1]].x, neighbors[move[1]].y)] = neighbors[move[1]]
        current = neighbors[move[1]]
      elif isCellOpen(neighbors[move[2]], n, currentGrid) and get3 is None:
        gridObj.allCells[(neighbors[move[2]].x, neighbors[move[2]].y)] = neighbors[move[2]]
        current = neighbors[move[2]]
      elif isCellOpen(neighbors[move[3]], n, currentGrid) and get4 is None: 
        gridObj.allCells[(neighbors[move[3]].x, neighbors[move[3]].y)] = neighbors[move[3]]
        current = neighbors[move[3]]
      else:
        if (current.x, current.y) == prev:
          return False
        prev = (current.x, current.y)
        pass
    path = reconstructPath(current, start)
    return executePlannedPath(path)

  def reconstructPath(current, start):
    #print('Reconstructing')
    path = collections.deque() 
    while True:
      if (current.x, current.y) == (start.x, start.y):
        break
      path.appendleft(current)
      current = gridObj.allCells[(current.parent[0], current.parent[1])]
    return path

  def executePlannedPath(path):
    #print('Executing')
    grid = gridObj.world
    goal = gridObj.goal
    isPathFeasible = True

    for index, current in enumerate(path):
      #print("Visiting ", current)
      gridObj.iterations += 1
      print(current.x, current.y)

      #print(current.x-current.parent[0], current.y-current.parent[1])
      if current.x == goal.x and current.y == goal.y:
        return gridObj.path

      if grid[current.x][current.y] == 1:
        knowledgeGrid.world[current.x][current.y] = 1
        isPathFeasible = False

        # this should be the restart point now
        setRestartPoint(gridObj.allCells[current.parent[0], current.parent[1]], 0)
        return None

      else:
        gridObj.path.append(current)

      # update the neighbours in the actual grid   
      # update the blockedNeighboursCount   
      if isCellWithinGrid(current.x + 1, current.y, n) and grid[current.x + 1][current.y] == 1:
        knowledgeGrid[current.x + 1, current.y] = 1
      
      if isCellWithinGrid(current.x + 1, current.y, n) and grid[current.x + 1][current.y] == 1:
        knowledgeGrid[current.x][current.y + 1] = 1
      
      if isCellWithinGrid(current.x - 1, current.y, n) and grid[current.x - 1][current.y] == 1:
        knowledgeGrid[current.x + 1][current.y] = 1
      
      if isCellWithinGrid(current.x - 1, current.y, n) and grid[current.x][current.y - 1] == 1:
        knowledgeGrid[current.x + 1][current.y] = 1
  
  def setRestartPoint(restartPoint, restartValue):
    for i in range(0, restartValue, 1):
      restartPoint = gridObj.allCells[(restartPoint.parent[0], restartPoint.parent[1])]

    gridObj.start = restartPoint
    return

  def printGridInItsFullGlory(n, world):
    for i in range(0, n, 1):
      for j in range(0, n, 1):
        print(world[i, j], end=' ')
      print()

  def IsGridSolvable(currentGrid, start):
    queue = PriorityQueue()
    closedMap = {}
    closedMap[(start.x, start.y)] = start
    start.parent = (start.x, start.y)
    queue.put((start.total_cost,start))    # first value in the tuple is priority(cost), second is the cell object

    goal = knowledgeGrid.goal

    while not queue.empty():   #while queue is not empty
      getter = queue.get()
      current = getter[1]
    
      rightCell = Cell(current.x + 1, current.y, current.cost_from_start + 1, 0, (current.x, current.y))     # right neighbour
      downCell = Cell(current.x, current.y + 1, current.cost_from_start + 1, 0, (current.x, current.y))    # down neighbour
      leftCell = Cell(current.x - 1, current.y, current.cost_from_start + 1, 0, (current.x, current.y))  # left neighbour
      topCell = Cell(current.x, current.y - 1, current.cost_from_start + 1, 0, (current.x, current.y))  # top neighbour
     
      neighbors = [rightCell, downCell, leftCell, topCell] # calculate the f(n) for each neighbour of current

      for i in neighbors:
        i.total_cost = calculatefn(i)         # calculate the f(n) for each neighbour of current
        isItOpen = isCellOpen(i, n, currentGrid)
        if isCellOpen(i,n, currentGrid):
          element = gridObj.allCells.get((i.x, i.y))
          if element is not None:
            ''' 
            If new path to neigbour costs less than previous path to it, then update cell.
            This cell has already been visited before, but we are just checking
            if the current path is less costlier than the one visited before
            '''
            if i.total_cost < gridObj.allCells[(i.x, i.y)].total_cost:
             gridObj.allCells[(i.x, i.y)] = i

          else:
           gridObj.allCells[(i.x, i.y)] = i      # this cell is being added first time
           queue.put((i.total_cost, i))

      if current.x == goal.x and current.y == goal.y: # To Do: Check if it's the goal and then update the goal if not
            return True
    return False

  def generateSolvableGrid():
    is_grid_solvable = False
    while not is_grid_solvable:
      gridObj = Grid(gridworld2(n,p), Cell(0, 0, 0, (n*2)-2, (0,0)), Cell(n-1,n-1,0,0,(0,0)), [])
      #check if grid is solvable
      is_grid_solvable = IsGridSolvable(gridObj.world,  Cell(0, 0, 0, (n*2)-2, (0,0)))
      
    knowledgeGrid = Grid(gridworld2(n,0), Cell(0, 0, 0, (n*2)-2, (0,0)), Cell(n-1,n-1,0,0,(0,0)), [])

  def repeatedNN():
    generateSolvableGrid()
    #printGridInItsFullGlory(n, gridObj.world)
    while True:
        if len(gridObj.path) > 0:
          previousStartPoint = gridObj.path[len(gridObj.path)-1]

          # if we are restarting from the same point where we restarted previously, then dont add this to the path again
          if gridObj.start.x != previousStartPoint.x and gridObj.start.y != previousStartPoint.y:
            gridObj.path.append(gridObj.start)

        path = NN(knowledgeGrid.world, gridObj.start)
        if path is None:
          continue

        elif path == False:
          return False

        # final path to goal found
        else:
          gridObj.world[0][0] = 5
          gridObj.world[gridObj.start.x][gridObj.start.y] = 5
          
          for i in gridObj.path:
            gridObj.world[i.x][i.y] = 5

          gridObj.world[gridObj.goal.x][gridObj.goal.y] = 5
          #print(gridObj.world)
          #printGridInItsFullGlory()
          print("Total iterations = ", gridObj.iterations)
          return True

  return repeatedNN()

NeuralVoyage(101, 0.0)

Starting at:  (0, 0)
(0, 0)
(0, 1)
(1, 1)
(1, 2)
(1, 3)
(2, 3)
(2, 4)
(2, 5)
(2, 6)
(2, 7)
(2, 8)
(3, 8)
(3, 9)
(3, 10)
(3, 11)
(4, 11)
(5, 11)
(5, 12)
(5, 13)
(5, 14)
(5, 15)
(5, 16)
(5, 17)
(5, 18)
(6, 18)
(6, 19)
(6, 20)
(6, 21)
(6, 22)
(6, 23)
(7, 23)
(7, 24)
(8, 24)
(8, 25)
(9, 25)
(9, 26)
(9, 27)
(9, 28)
(9, 29)
(9, 30)
(10, 30)
(10, 31)
(10, 32)
(11, 32)
(11, 33)
(11, 34)
(12, 34)
(12, 35)
(13, 35)
(14, 35)
(15, 35)
(15, 36)
(15, 37)
(15, 38)
(15, 39)
(15, 40)
(16, 40)
(17, 40)
(17, 41)
(18, 41)
(18, 42)
(18, 43)
(19, 43)
(19, 44)
(19, 45)
(19, 46)
(19, 47)
(20, 47)
(20, 48)
(20, 49)
(20, 50)
(20, 51)
(21, 51)
(21, 52)
(22, 52)
(22, 53)
(22, 54)
(22, 55)
(22, 56)
(23, 56)
(23, 57)
(24, 57)
(24, 58)
(25, 58)
(25, 59)
(26, 59)
(26, 60)
(26, 61)
(26, 62)
(27, 62)
(27, 63)
(28, 63)
(29, 63)
(30, 63)
(30, 64)
(30, 65)
(30, 66)
(30, 67)
(30, 68)
(30, 69)
(30, 70)
(31, 70)
(31, 71)
(31, 72)
(31, 73)
(32, 73)
(32, 74)
(33, 74)
(33, 75)
(33, 76)
(33, 77)
(34, 77)
(34, 78)
(35, 78)
(35, 7

False

In [ ]:
listd = [1,2,3,4]
max = listd.index(np.max(listd))
listd.pop(max)
print(np.max(listd))

3


# Convolutional Neural Network for Project 1

In [ ]:
train_in = np.reshape( train_in, (-1, 101, 101, 1) )
#test_in = np.reshape( test_in, (-1, 101, 101, 1) )

grid_input = tf.keras.layers.Input( shape = (101,101, 1) )

#maybe kernel size should be 9,9
cnn_1 = tf.keras.layers.Conv2D( filters = 64, kernel_size = (3,3), strides = (1,1),
                               padding = "valid", activation = tf.nn.relu )( grid_input )

#cnn_2 = tf.keras.layers.Conv2D( filters = 64, kernel_size = (9,9), strides = (1,1),
#                               padding = "valid", activation = tf.nn.relu )( cnn_1 )

flatten_image = tf.keras.layers.Flatten()( cnn_1 )
dropout_1 = tf.keras.layers.Dropout( rate = 0.5 )( flatten_image )
dense_1 = tf.keras.layers.Dense( units = 1000, activation = tf.nn.relu )( dropout_1 )
#dense_2 = tf.keras.layers.Dense( units = 300, activation = tf.nn.relu )( dense_1 )

movement_output = tf.keras.layers.Dense( units = 4, activation = None )( dense_1 )
probabilities = tf.keras.layers.Softmax()( movement_output )

conv_model = tf.keras.Model( inputs = grid_input, outputs = probabilities )
conv_model.compile( optimizer = 'adam', loss = 'categorical_crossentropy', metrics = ['accuracy'] )

conv_model.fit(train_in, train_out, epochs = 5)
#print(history)
conv_model.predict(train_in[0:1])

ResourceExhaustedError: ignored

# Neural network to mimic Project 2

In [ ]:
Project2MovementFrame = pd.DataFrame(Project2MovementDict)
print(Project2MovementFrame)
Grids = np.asarray(Project2MovementDict['MinusInferance'])
SensedGrids = np.asarray(Project2MovementDict['SensedNeighboursGrid'])
y_train_2 = np.asarray(Project2MovementDict['Movement'])

print('Grids shape = ', Grids.shape)
print('SensedGrids shape = ', SensedGrids.shape)

Grids = Grids.reshape(len(Grids), 101, 101)
SensedGrids = SensedGrids.reshape(len(Grids), 101, 101)

print('Grids shape = ', Grids.shape)
print('SensedGrids shape = ', SensedGrids.shape)

x_train_2 = np.concatenate([Grids,SensedGrids], axis = 1)
print('Shape of Inputs', x_train_2.shape)

print('Number of grids =', len(Project2MovementDict['MinusInferance']))
#print('x_train')
print('size of x_train',x_train_2.shape)
#print(x_train)

#print(y_train_2)
print('shape of y_train', y_train_2.shape)

#shape prints rows, columns
#project 2 has two 101*101 grids one on top of another
#rows = 101 + 101 = 202
#columns = 101

#x_train, x_test, y_train, y_test = train_test_split(X, Y, test_size=0.2, random_state =1)

train_in = np.reshape( x_train_2, (-1, 202, 101) )
train_out = np.reshape( y_train_2, (-1, 4) )

#define neural network layers, size of each layer (no. of nodes in each layer)
grid_input = tf.keras.layers.Input(shape = (202,101))
flatten_image = tf.keras.layers.Flatten()( grid_input )
#dense_1 is the first hidden layer (fully connected)
dense_1 = tf.keras.layers.Dense( units = 1000, activation = tf.nn.relu )( flatten_image )

#dense_2 is the second hidden layer (fully connected)
dense_2 = tf.keras.layers.Dense( units = 800, activation = tf.nn.relu )( dense_1 )

#output layer
movement_output = tf.keras.layers.Dense( units = 4, activation = None )( dense_2 )

probabilities = tf.keras.layers.Softmax()( movement_output )

model = tf.keras.Model( inputs = grid_input, outputs = probabilities )
model.compile( optimizer = 'adam', loss = 'categorical_crossentropy', metrics = ['accuracy'] )

model.fit( train_in, train_out, epochs = 5)
#print(history)
model.predict(train_in[0:1])

# Convolutional Neural Network for Project 2

In [ ]:
train_in = np.reshape( train_in, (-1, 202, 101, 1) )
#test_in = np.reshape( test_in, (-1, 101, 101, 1) )

grid_input = tf.keras.layers.Input( shape = (202,101, 1) )

#maybe kernel size should be 9,9
cnn_2 = tf.keras.layers.Conv2D( filters = 64, kernel_size = (3,3), strides = (1,1),
                               padding = "valid", activation = tf.nn.relu )( grid_input )

#cnn_2 = tf.keras.layers.Conv2D( filters = 64, kernel_size = (9,9), strides = (1,1),
#                               padding = "valid", activation = tf.nn.relu )( cnn_1 )

flatten_image = tf.keras.layers.Flatten()( cnn_2 )
dropout_1 = tf.keras.layers.Dropout( rate = 0.5 )( flatten_image )
dense_1 = tf.keras.layers.Dense( units = 800, activation = tf.nn.relu )( dropout_1 )
#dense_2 = tf.keras.layers.Dense( units = 300, activation = tf.nn.relu )( dense_1 )

movement_output = tf.keras.layers.Dense( units = 4, activation = None )( dense_1 )
probabilities = tf.keras.layers.Softmax()( movement_output )

conv_model = tf.keras.Model( inputs = grid_input, outputs = probabilities )
conv_model.compile( optimizer = 'adam', loss = 'categorical_crossentropy', metrics = ['accuracy'] )

conv_model.fit(train_in, train_out, epochs = 5)
#print(history)
conv_model.predict(train_in[0:1])

# Neural Voyage without planning

In [ ]:
def NeuralVoyage(n,p):
  class Cell:
    # x = x-coordinate, y = y-coordinate in grid. Values from 0 to n-1
    def __init__(self, x, y,cost_from_start, total_cost, parent):
      self.x = x
      self.y = y
      self.cost_from_start = cost_from_start
      self.total_cost = total_cost
      self.parent = parent

    def __lt__(self, other):    # Added to add an object to the priority queue
        return self.total_cost < other.total_cost

  class Grid:
    def __init__(self, world, start, goal, path):
      self.iterations = 0
      self.world = world
      self.start = start
      self.goal = goal
      self.path = []
      self.restartValue = 0
      self.allCells = {}    # a dictionary to lookup a cell. Used when looking up its gScore

  def gridworld2(n,p):
    start = [0,0]; end = [n-1,n-1]
    gridworld = []

    for width in range(n):    # Loop creates n lists of n elements
      width = []
      for length in range(n):
        rand = np.random.random()
        if rand > p:
          width.append(0)
        else:
          width.append(1)
      gridworld.append(width)

    gridworld = np.array(gridworld)    # List of lists is turned into an array
    gridworld[start] = 0; gridworld[end] = 0
    return gridworld
  
  gridObj = Grid(gridworld2(n,p), Cell(0, 0, 0, (n*2)-2, (0,0)), Cell(n-1,n-1,0,0,(0,0)), [])
  knowledgeGrid = Grid(gridworld2(n,0), Cell(0, 0, 0, (n*2)-2, (0,0)), Cell(n-1,n-1,0,0,(0,0)), [])


  def isCellWithinGrid(x, y, n):
    if (x < 0) or (x > (n - 1)) or (y < 0) or (y > (n - 1)):
      return False

  def isCellOpen(cell, n, grid):
    if isCellWithinGrid(cell.x, cell.y, n) == False: # check if within gridworld bounds
      return False
    
    if grid[cell.x][cell.y] == 1:     # check if blocked
      return False
    
    return True

  def Direct(currentGrid, current):
      world = currentGrid
      world2 = world
      world[current.x,current.y]=2
      world[knowledgeGrid.goal.x,knowledgeGrid.goal.y]=3
      #print(world)
      world = world.flatten()
      test_in = np.reshape(world, (-1, 101, 101) )
      predicted_move = model.predict(test_in)
      #print(predicted_move)
      predicted_move = predicted_move[0].tolist()
      direction = predicted_move.index(np.max(predicted_move))
      fourth = predicted_move.index(np.min(predicted_move))
      predicted_move[direction] = 0
      second = predicted_move.index(np.max(predicted_move))
      predicted_move[second] = 0
      third = predicted_move.index(np.max(predicted_move))
      predicted_move[third] = 0
      world = world2
      world[current.x,current.y]=0
      #print(direction, second, third, fourth)
      return (direction, second, third, fourth)
      
  # this is the planning phase  
  def NN(currentGrid, start):
    gridObj.allCells = {}
    gridObj.allCells[(start.x, start.y)] = start
    print('Starting at: ', (start.x, start.y))
    current = start
    goal = gridObj.goal
    while (current.x, current.y) != (goal.x, goal.y):
      print((current.x, current.y))
      rightCell = Cell(current.x + 1, current.y, current.cost_from_start + 1, 0,
                      (current.x, current.y))  # right neighbour
      downCell = Cell(current.x, current.y + 1, current.cost_from_start + 1, 0,
                      (current.x, current.y))  # down neighbour
      leftCell = Cell(current.x - 1, current.y, current.cost_from_start + 1, 0,
                      (current.x, current.y))  # left neighbour
      topCell = Cell(current.x, current.y - 1, current.cost_from_start + 1, 0,
                      (current.x, current.y))  # top neighbour

      neighbors = [rightCell, leftCell, topCell, downCell]  # calculate the f(n) for each neighbour of current
      move = Direct(knowledgeGrid.world, current)
      get1 = gridObj.allCells.get((neighbors[move[0]].x, neighbors[move[0]].y))
      get2 = gridObj.allCells.get((neighbors[move[1]].x, neighbors[move[1]].y))
      get3 = gridObj.allCells.get((neighbors[move[2]].x, neighbors[move[2]].y))
      get4 = gridObj.allCells.get((neighbors[move[3]].x, neighbors[move[3]].y))
      if isCellOpen(neighbors[move[0]], n, currentGrid) and get1 is None:
        gridObj.allCells[(neighbors[move[0]].x, neighbors[move[0]].y)] = neighbors[move[0]]
        current = neighbors[move[0]]
      elif isCellOpen(neighbors[move[1]], n, currentGrid) and get2 is None:
        gridObj.allCells[(neighbors[move[1]].x, neighbors[move[1]].y)] = neighbors[move[1]]
        current = neighbors[move[1]]
      elif isCellOpen(neighbors[move[2]], n, currentGrid) and get3 is None:
        gridObj.allCells[(neighbors[move[2]].x, neighbors[move[2]].y)] = neighbors[move[2]]
        current = neighbors[move[2]]
      elif isCellOpen(neighbors[move[3]], n, currentGrid) and get4 is None: 
        gridObj.allCells[(neighbors[move[3]].x, neighbors[move[3]].y)] = neighbors[move[3]]
        current = neighbors[move[3]]
      else:
        path = reconstructPath(current, start)
        return executePlannedPath(path)
    path = reconstructPath(current, start)
    return executePlannedPath(path)

  def reconstructPath(current, start):
    print('Reconstructing')
    path = collections.deque() 
    while True:
      if (current.x, current.y) == (start.x, start.y):
        break
      path.appendleft(current)
      current = gridObj.allCells[(current.parent[0], current.parent[1])]
    return path

  def executePlannedPath(path):
    print('Executing')
    grid = gridObj.world
    goal = gridObj.goal
    isPathFeasible = True

    for index, current in enumerate(path):
      #print("Visiting ", current)
      gridObj.iterations += 1
      print(current.x, current.y)

      #print(current.x-current.parent[0], current.y-current.parent[1])
      if current.x == goal.x and current.y == goal.y:
        return gridObj.path

      if grid[current.x][current.y] == 1:
        knowledgeGrid.world[current.x][current.y] = 1
        isPathFeasible = False

        # this should be the restart point now
        setRestartPoint(gridObj.allCells[current.parent[0], current.parent[1]], 0)
        return None

      else:
        gridObj.path.append(current)

      # update the neighbours in the actual grid   
      # update the blockedNeighboursCount   
      if isCellWithinGrid(current.x + 1, current.y, n) and grid[current.x + 1][current.y] == 1:
        knowledgeGrid[current.x + 1, current.y] = 1
      
      if isCellWithinGrid(current.x + 1, current.y, n) and grid[current.x + 1][current.y] == 1:
        knowledgeGrid[current.x][current.y + 1] = 1
      
      if isCellWithinGrid(current.x - 1, current.y, n) and grid[current.x - 1][current.y] == 1:
        knowledgeGrid[current.x + 1][current.y] = 1
      
      if isCellWithinGrid(current.x - 1, current.y, n) and grid[current.x][current.y - 1] == 1:
        knowledgeGrid[current.x + 1][current.y] = 1
  
  def setRestartPoint(restartPoint, restartValue):
    for i in range(0, restartValue, 1):
      restartPoint = gridObj.allCells[(restartPoint.parent[0], restartPoint.parent[1])]

    gridObj.start = restartPoint
    return

  def makeSingleMove(currentGrid, start):
    currentGrid[start.x, start.y] = 2
    predicted_move = model.predict(currentGrid)
    #world = world.flatten()
    #test_in = np.reshape(world, (-1, 101, 101) )
    move = []
    
    #sorts in ascending
    sorted_probabilities = np.argsort(predicted_move)

    move[0] = predicted_move[sorted_probabilities[3]]
    move[1] = predicted_move[sorted_probabilities[2]]
    move[2] = predicted_move[sorted_probabilities[1]]

    
    


  def repeatedNN():
    while True:
        if len(gridObj.path) > 0:
          previousStartPoint = gridObj.path[len(gridObj.path)-1]

          # if we are restarting from the same point where we restarted previously, then dont add this to the path again
          if gridObj.start.x != previousStartPoint.x and gridObj.start.y != previousStartPoint.y:
            gridObj.path.append(gridObj.start)

        path = NN(knowledgeGrid.world, gridObj.start)
        if path is None:
          continue

        elif path == False:
          return False

        # final path to goal found
        else:
          gridObj.world[0][0] = 5
          gridObj.world[gridObj.start.x][gridObj.start.y] = 5
          
          for i in gridObj.path:
            gridObj.world[i.x][i.y] = 5

          gridObj.world[gridObj.goal.x][gridObj.goal.y] = 5
          print(gridObj.world)
          print("Total iterations = ", gridObj.iterations)
          return True

  return repeatedNN()

NeuralVoyage(101,0.1)

# Comparison

In [ ]:
def Comparison(n, p):
    class Cell:
        # x = x-coordinate, y = y-coordinate in grid. Values from 0 to n-1
        def __init__(self, x, y, cost_from_start, total_cost, parent):
            self.x = x
            self.y = y
            self.cost_from_start = cost_from_start
            self.total_cost = total_cost
            self.total_neighbours = 0
            self.total_blocked_neighbours = 0
            self.known_neighbours = 0
            self.known_blocked_neighbours = 0
            self.parent = parent
            self.confirmed = False
            self.visited = False

        def __lt__(self, other):  # Added to add an object to the priority queue
            return self.total_cost < other.total_cost

    class Grid:
        def __init__(self, world, start, goal, path):
            self.iterations = 0
            self.world = world
            self.start = start
            self.goal = goal
            self.path = []
            self.cellMap = {}  # a dictionary to lookup a cell.
            self.allCells = {}

    def gridworld2(n,p):
      start = [0,0]; end = [n-1,n-1]
      gridworld = []

      for width in range(n):    # Loop creates n lists of n elements
        width = []
        for length in range(n):
          rand = np.random.random()
          if rand > p:
            width.append(0)
          else:
            width.append(1)
        gridworld.append(width)

      gridworld = np.array(gridworld)    # List of lists is turned into an array
      gridworld[start] = 0; gridworld[end] = 0
      return gridworld
    
    g = gridworld2(n,p)
    gridObj = Grid(g, Cell(0, 0, 0, (n*2)-2, (0,0)), Cell(n-1,n-1,0,0,(0,0)), [])
    gridObj2 = Grid(g, Cell(0, 0, 0, (n*2)-2, (0,0)), Cell(n-1,n-1,0,0,(0,0)), []) 
    gridObj3 = Grid(g, Cell(0, 0, 0, (n*2)-2, (0,0)), Cell(n-1,n-1,0,0,(0,0)), []) 
    k = gridworld2(n,0)
    knowledgeGrid = Grid(k, Cell(0, 0, 0, (n * 2) - 2, (0, 0)), Cell(n - 1, n - 1, 0, 0, (0, 0)), [])
    knowledgeGrid2 = Grid(k, Cell(0, 0, 0, (n * 2) - 2, (0, 0)), Cell(n - 1, n - 1, 0, 0, (0, 0)), [])
    knowledgeGrid3 = Grid(k, Cell(0, 0, 0, (n * 2) - 2, (0, 0)), Cell(n - 1, n - 1, 0, 0, (0, 0)), [])
    KnowledgeGridMinusInferance = Grid(gridworld2(n, 0), Cell(0, 0, 0, (n * 2) - 2, (0, 0)), Cell(n - 1, n - 1, 0, 0, (0, 0)), []) #declaration
    InferanceGrid = Grid(gridworld2(n, 0), Cell(0, 0, 0, (n * 2) - 2, (0, 0)), Cell(n - 1, n - 1, 0, 0, (0, 0)), []) #declaration
    visited_grid = gridworld2(n, 0)

    def isCellWithinGrid(x, y, n):
      if (x < 0) or (x > (n - 1)) or (y < 0) or (y > (n - 1)):
        return False

    def isCellOpen(cell, n, grid):
      if isCellWithinGrid(cell.x, cell.y, n) == False: # check if within gridworld bounds
        return False
      
      if grid[cell.x][cell.y] == 1:    # check if blocked
        return False
      
      return True

    def calculatefn(cell):
      goal = (n - 1, n - 1)
      estimate = calculateManhattanDistance(cell, goal)
      return estimate + cell.cost_from_start

    def calculateManhattanDistance(cell, goal):
      return abs(cell.x - goal[0]) + abs(cell.y - goal[1])

############### Project 1 Agent ##################################

    def Voyage(n,p):
      # this is the planning phase
      def AStar(currentGrid, start):
        queue = PriorityQueue()
        gridObj.allCells = {}
        gridObj.allCells[(start.x, start.y)] = start
        queue.put((start.total_cost,start))    # first value in the tuple is priority(cost), second is the cell object
        goal = gridObj.goal

        while not queue.empty():   #while queue is not empty
          getter = queue.get()
          current = getter[1]

          if current.x == goal.x and current.y == goal.y:
            path = reconstructPath(current, start)
            return executePlannedPath(path) 
        
          rightCell = Cell(current.x + 1, current.y, current.cost_from_start + 1, 0, (current.x, current.y))     # right neighbour
          downCell = Cell(current.x, current.y + 1, current.cost_from_start + 1, 0, (current.x, current.y))    # down neighbour
          leftCell = Cell(current.x - 1, current.y, current.cost_from_start + 1, 0, (current.x, current.y))  # left neighbour
          topCell = Cell(current.x, current.y - 1, current.cost_from_start + 1, 0, (current.x, current.y))  # top neighbour
        
          neighbors = [rightCell, downCell, leftCell, topCell] # calculate the f(n) for each neighbour of current

          for i in neighbors:
            i.total_cost = calculatefn(i)         # calculate the f(n) for each neighbour of current
            isItOpen = isCellOpen(i, n, currentGrid)
            if isCellOpen(i,n, currentGrid):
              element = gridObj.allCells.get((i.x, i.y))
              if element is not None:
                ''' 
                If new path to neigbour costs less than previous path to it, then update cell.
                This cell has already been visited before, but we are just checking
                if the current path is less costlier than the one visited before
                '''
                if i.total_cost < gridObj.allCells[(i.x, i.y)].total_cost:
                  gridObj.allCells[(i.x, i.y)] = i

              else:
                gridObj.allCells[(i.x, i.y)] = i      # this cell is being added first time
                queue.put((i.total_cost, i))
        
        # queue is empty but goal node has not been reached
        return False


      def reconstructPath(current, start):
        path = collections.deque()
        
        while True:
          if (current.x, current.y) == (start.x, start.y):
            break
          path.appendleft(current)
          current = gridObj.allCells[(current.parent[0], current.parent[1])]
        
        return path

      def executePlannedPath(path):
        grid = gridObj.world
        goal = gridObj.goal
        isPathFeasible = True

        for index, current in enumerate(path):
          #print("Visiting ", current)
          gridObj.iterations += 1
          if current.x == goal.x and current.y == goal.y:
            MovementDict['Movement'].append([0,0,0,1])
            return gridObj.path

          if grid[current.x][current.y] == 1:
            knowledgeGrid.world[current.x][current.y] = 1
            isPathFeasible = False

            # this should be the restart point now
            setRestartPoint(gridObj.allCells[current.parent[0], current.parent[1]], 0)
            return None

          else:
            gridObj.path.append(current)

          # update the neighbours in the actual grid   
          # update the blockedNeighboursCount   
          if isCellWithinGrid(current.x + 1, current.y, n) and grid[current.x + 1][current.y] == 1:
            knowledgeGrid[current.x + 1, current.y] = 1
          
          if isCellWithinGrid(current.x + 1, current.y, n) and grid[current.x + 1][current.y] == 1:
            knowledgeGrid[current.x][current.y + 1] = 1
          
          if isCellWithinGrid(current.x - 1, current.y, n) and grid[current.x - 1][current.y] == 1:
            knowledgeGrid[current.x + 1][current.y] = 1
          
          if isCellWithinGrid(current.x - 1, current.y, n) and grid[current.x][current.y - 1] == 1:
            knowledgeGrid[current.x + 1][current.y] = 1
      
      def setRestartPoint(restartPoint, restartValue):
        for i in range(0, restartValue, 1):
          restartPoint = gridObj.allCells[(restartPoint.parent[0], restartPoint.parent[1])]

        gridObj.start = restartPoint
        return

      def restartController(current):
        # in a loop run the restart again and again
        # starting from the current cell's parent to further up
        while (current.x != 0 and current.y != 0):
          current = gridObj.allCells[(current.parent[0], current.parent[1])]
          gridObj.start = current

      def repeatedAStar():
        while True:
            if len(gridObj.path) > 0:
              previousStartPoint = gridObj.path[len(gridObj.path)-1]

              # if we are restarting from the same point where we restarted previously, then dont add this to the path again
              if gridObj.start.x != previousStartPoint.x and gridObj.start.y != previousStartPoint.y:
                gridObj.path.append(gridObj.start)

            path = AStar(knowledgeGrid.world, gridObj.start)
            if path is None:
              continue

            elif path == False:
              return False

            # final path to goal found
            else:
              leng = 2
              knowledgeGrid.world[0][0] = 5
              knowledgeGrid.world[gridObj.start.x][gridObj.start.y] = 5
              
              for i in gridObj.path:
                leng += 1
                knowledgeGrid.world[i.x][i.y] = 5

              knowledgeGrid.world[gridObj.goal.x][gridObj.goal.y] = 5
              print(gridObj.world)
              print(knowledgeGrid.world)
              print("Total iterations = ", gridObj.iterations)
              return (True, gridObj.iterations, leng)
        
      def restartAtDifferentPoints():
        for i in range(0, (n*n - 1)):
          repeatedAStar()
        return

      return repeatedAStar()

################## Project 2 Agent ###############################

    def PartialSensing(n, p):
        class Solver:
            def __init__(self):
                # a variable is a cell
                # constraints are all the equations that this cell is part of
                self.variables_to_constraints_map = {}

                # each constraint will a key value pair
                # key is a tuple, blocked_neighbours is a value
                # this is a list of constraints
                self.constraints = []

            def add_constraint(self, variable, n_list, value):
                variable_set = set()
                for i in n_list:
                    variable_set.add((i.x * n + i.y))

                constraint = {}
                constraint[tuple(variable_set)] = value
                self.constraints.append(constraint)
                constraintIndex = len(self.constraints) - 1

                # adding constraint
                for v in variable_set:
                    if self.variables_to_constraints_map.get(v) is None:
                        self.variables_to_constraints_map[v] = []
                    self.variables_to_constraints_map[v].append(constraintIndex)

            def check_satisfaction_conditions(self, lhs, rhs):
                if rhs == 0:
                    # all cells on lhs are free
                    for i in range(0, len(lhs), 1):
                        cellX, cellY = divmod(lhs[i], n)
                        inferred_free(cellX, cellY)

                elif len(lhs) == rhs:
                    # all cells on lhs are blocked
                    for i in range(0, len(lhs), 1):
                        cellX, cellY = divmod(lhs[i], n)
                        inferred_blocked(cellX, cellY)

            def solve_constraint(self, cell, cell_value):
                # Solved when:
                # RHS becomes zero means all are free
                # Size of the list/set on left side = no. on the right side means all are blocked or all are free
                variable = cell.x * n + cell.y
                # get all constraint equations of this variable
                if self.variables_to_constraints_map.get(variable) is None:
                    return

                constraints = self.variables_to_constraints_map[variable]
                for constraint_index in constraints:
                    constraint = self.constraints[constraint_index]
                    (lhs, rhs), = constraint.items()
                    temp_lhs = list(lhs)
                    temp_lhs.remove(variable)
                    lhs = tuple(temp_lhs)

                    if cell_value == 1:
                        rhs -= 1

                    self.variables_to_constraints_map[variable].remove(constraint_index)
                    constraint = {}
                    constraint[lhs] = rhs
                    self.constraints[constraint_index] = constraint
                    self.check_satisfaction_conditions(lhs, rhs)
                return

        solver = Solver()

        def inferred_free(x, y):
            if knowledgeGrid2.cellMap[(x, y)].confirmed:
                return
            knowledgeGrid2.world[x, y] = 0
            knowledgeGrid2.cellMap[(x, y)].confirmed = True
            # inform neighbours that this cell has been free
            informNeigboursAboutNewFreeCell(knowledgeGrid2.cellMap[(x, y)])
            solver.solve_constraint(knowledgeGrid2.cellMap[(x, y)], 0)

        def inferred_blocked(x, y):
            if knowledgeGrid2.cellMap[(x, y)].confirmed:
                return

            knowledgeGrid2.world[x, y] = 1
            knowledgeGrid2.cellMap[(x, y)].confirmed = True
            # inform neighbours that this cell has been free
            informNeighboursAboutNewBlockedCell(knowledgeGrid2.cellMap[(x, y)])
            solver.solve_constraint(knowledgeGrid2.cellMap[(x, y)], 1)

        def agent_four_inference(cell, unknown_blocked):
            neighbours = getNeighbours(cell)
            for i in neighbours:
                if i.confirmed:
                    if knowledgeGrid2.world[i.x, i.y] == 1:
                        unknown_blocked -= 1
                    # remove this from the equation because its value is known
                    # if the value was 1, then that case was handled in the above line
                    # if value was 0, then there is no effect on RHS
                    neighbours.remove(i)

            solver.add_constraint(cell.x * n + cell.y, neighbours, unknown_blocked)
            return

        # Probability Start
        def calculateProbilityfn(cell):
            goal = (n - 1, n - 1)

            # when some neighbours are unknown
            if (cell.total_neighbours - cell.known_neighbours) != 0:
                estimateWithProbability = round(calculateManhattanDistance(cell, goal) * (((
                                                                                                      cell.total_blocked_neighbours - cell.known_blocked_neighbours) / (
                                                                                                      cell.total_neighbours - cell.known_neighbours)) + 1))

            # when all neighbours of a cell are known, no need to involve probabilistic calculations for them
            else:
                estimateWithProbability = calculateManhattanDistance(cell, goal)
            # print('Estimate : ' + str(estimateWithProbability))
            return estimateWithProbability + cell.cost_from_start
            # Probability End

        def getNeighbours(cell):
            # east, south, west, north, ne, se, sw, nw in that order
            x = [1, 0, -1, 0, 1, 1, -1, -1]
            y = [0, 1, 0, -1, -1, 1, 1, -1]

            neighbours = []
            #the parent being added here is wrong
            #this thing is resetting the parent
            #
            for i in range(8):
                if isCellWithinGrid(cell.x + x[i], cell.y + y[i], n):
                    if knowledgeGrid2.cellMap.get((cell.x + x[i], cell.y + y[i])) is None:
                        knowledgeGrid2.cellMap[(cell.x + x[i], cell.y + y[i])] = Cell(cell.x + x[i], cell.y + y[i], 0, 0,
                                                                                    (cell.x, cell.y))
                    neighbours.append(knowledgeGrid2.cellMap[(cell.x + x[i], cell.y + y[i])])

            return neighbours

        def senseNeighbours(cell):
            ECell = Cell(cell.x + 1, cell.y, cell.cost_from_start + 1, 0, (cell.x, cell.y))  # East neighbour
            SCell = Cell(cell.x, cell.y + 1, cell.cost_from_start + 1, 0, (cell.x, cell.y))  # South neighbour
            WCell = Cell(cell.x - 1, cell.y, cell.cost_from_start + 1, 0, (cell.x, cell.y))  # West neighbour
            NCell = Cell(cell.x, cell.y - 1, cell.cost_from_start + 1, 0, (cell.x, cell.y))  # North neighbour
            NECell = Cell(cell.x + 1, cell.y - 1, cell.cost_from_start + 1, 0, (cell.x, cell.y))  # North East neighbour
            SECell = Cell(cell.x + 1, cell.y + 1, cell.cost_from_start + 1, 0, (cell.x, cell.y))  # South East neighbour
            SWCell = Cell(cell.x - 1, cell.y + 1, cell.cost_from_start + 1, 0, (cell.x, cell.y))  # South West neighbour
            NWCell = Cell(cell.x - 1, cell.y - 1, cell.cost_from_start + 1, 0, (cell.x, cell.y))  # North West neighbour

            potential_neighbours = [ECell, SCell, WCell, NCell, NECell, SECell, SWCell, NWCell]
            neighbours = [i for i in potential_neighbours if isCellWithinGrid(i.x, i.y, n)]
            total = 0
            blocked = 0
            for i in neighbours:
                total += 1
                if gridObj.world[i.x, i.y] == 1:
                    blocked += 1
            cell.total_neighbours = total
            cell.total_blocked_neighbours = blocked
            return cell

        def getKnownNeighbours(cell):
            # east, south, west, north, ne, se, sw, nw in that order
            x = [1, 0, -1, 0, 1, 1, -1, -1]
            y = [0, 1, 0, -1, -1, 1, 1, -1]

            neighbours = []
            for i in range(8):
                if isCellWithinGrid(cell.x + x[i], cell.y + y[i], n):
                    neighbourCell = knowledgeGrid2.cellMap.get((cell.x + x[i], cell.y + y[i]))
                    if neighbourCell is not None and neighbourCell.confirmed:
                        cell.known_neighbours += 1
                        if knowledgeGrid2.world[cell.x + x[i], cell.y + y[i]] == 1:
                            cell.known_blocked_neighbours += 1
            return cell

        def updateKnownNeighbours(cell):
            knowledgeGrid2.cellMap[(cell.x, cell.y)].known_neighbours += 1
            inferenceEngine(cell)

        def updateKnownBlockedNeighbours(cell):
            cell.known_blocked_neighbours += 1
            cell.known_neighbours += 1
            knowledgeGrid2.cellMap[(cell.x, cell.y)] = cell
            inferenceEngine(cell)

        def inferenceEngine(cell):
            if cell.total_blocked_neighbours == cell.known_blocked_neighbours:
                markRemainingNeighboursAsFree(cell)

            # if totalFreeNeighbours = knownFreeNeighbours
            if (cell.total_neighbours - cell.total_blocked_neighbours
                    == (cell.known_neighbours - cell.known_blocked_neighbours)):
                markRemainingNeighboursAsBlocked(cell)

            # Early Exit Function
            if cell.total_blocked_neighbours == cell.total_neighbours - 1:
                markRemainingNeighboursAsBlocked(cell)

        def markRemainingNeighboursAsFree(cell):
            neighbours = getNeighbours(cell)
            for i in neighbours:
                # this new freely marked cell must inform its neighbours which have been visited
                if i.confirmed == False:
                    knowledgeGrid2.world[i.x, i.y] = 0
                    cell.known_neighbours += 1
                    knowledgeGrid2.cellMap[(i.x, i.y)] = cell
                    i.confirmed = True
                    knowledgeGrid2.cellMap[(i.x, i.y)] = i
                    # inform the neighbours
                    informNeigboursAboutNewFreeCell(i)
                    solver.solve_constraint(knowledgeGrid2.cellMap[(i.x, i.y)], 0)

        def markRemainingNeighboursAsBlocked(cell):
            neighbours = getNeighbours(cell)
            for i in neighbours:
                # this new blocked cell must inform its neighbours which have been visited
                if i.confirmed == False:
                    cell.known_blocked_neighbours += 1
                    cell.known_neighbours += 1
                    knowledgeGrid2.cellMap[(i.x, i.y)] = cell
                    knowledgeGrid2.world[i.x, i.y] = 1
                    i.confirmed = True
                    knowledgeGrid2.cellMap[(i.x, i.y)] = i
                    informNeighboursAboutNewBlockedCell(i)
                    solver.solve_constraint(knowledgeGrid2.cellMap[(i.x, i.y)], 1)

        def informNeigboursAboutNewFreeCell(cell):
            neighbours = getNeighbours(cell)
            for i in neighbours:
                if i.visited:
                    updateKnownNeighbours(i)

        def informNeighboursAboutNewBlockedCell(cell):
            neighbours = getNeighbours(cell)
            for i in neighbours:
                if i.visited:
                    updateKnownBlockedNeighbours(i)

        # this is the planning phase
        def AStar(knowledgeGrid, start):
            # print("Starting from ", start.x, start.y)
            queue = PriorityQueue()
            closedMap = {}
            closedMap[(start.x, start.y)] = start
            queue.put((start.total_cost, start))  # first value in the tuple is priority(cost), second is the cell object
            goal = knowledgeGrid2.goal

            while not queue.empty():  # while queue is not empty
                getter = queue.get()
                current = getter[1]
                if isCellOpen(current, n, knowledgeGrid2.world) is False and isCellWithinGrid(current.x, current.y, n) is True:
                    knowledgeGrid2.world[current.x, current.y] = 1

                if current.x == goal.x and current.y == goal.y:
                    path = reconstructPath(current, start, closedMap)
                    return executePlannedPath(path)

                rightCell = Cell(current.x + 1, current.y, current.cost_from_start + 1, 0,
                                (current.x, current.y))  # right neighbour
                downCell = Cell(current.x, current.y + 1, current.cost_from_start + 1, 0,
                                (current.x, current.y))  # down neighbour
                leftCell = Cell(current.x - 1, current.y, current.cost_from_start + 1, 0,
                                (current.x, current.y))  # left neighbour
                topCell = Cell(current.x, current.y - 1, current.cost_from_start + 1, 0,
                              (current.x, current.y))  # top neighbour

                neighbors = [rightCell, downCell, leftCell, topCell]  # calculate the f(n) for each neighbour of current

                for i in neighbors:
                    # i.total_cost = calculatefn(i)  # calculate the f(n) for each neighbour of current

                    if knowledgeGrid2.cellMap.get((i.x, i.y)) and knowledgeGrid2.cellMap[(i.x, i.y)].visited:
                        i.total_cost = calculateProbilityfn(knowledgeGrid2.cellMap[(i.x, i.y)])
                    else:
                        i.total_cost = calculatefn(i)
                        #knowledgeGrid.cellMap[(i.x, i.y)].parent = (current.x, current.y)

                    if isCellOpen(i, n, knowledgeGrid2.world):
                        element = closedMap.get((i.x, i.y))
                        if element is not None:
                            ''' 
                            If new path to neigbour costs less than previous path to it, then update cell.
                            This cell has already been visited before, but we are just checking
                            if the current path is less costlier than the one visited before
                            '''
                            if i.total_cost < closedMap[(i.x, i.y)].total_cost:
                                closedMap[(i.x, i.y)] = i

                        else:
                            closedMap[(i.x, i.y)] = i  # this cell is being added first time
                            queue.put((i.total_cost, i))
                previous = current
            # queue is empty but goal node has not been reached
            return False

        def reconstructPath(current, start, closedMap):
            path = collections.deque()

            while True:
                if (current.x, current.y) == (start.x, start.y):
                    break
                path.appendleft(current)
                current = closedMap[(current.parent[0], current.parent[1])]

            return path


        def executePlannedPath(path):
            grid = gridObj2.world
            goal = gridObj2.goal

            # print('total blocked at 00   ' + str(gridObj.cellMap[0, 0].total_blocked_neighbours))
            InferanceGrid.world[0, 0] = gridObj2.cellMap[0, 0].total_blocked_neighbours

            prev = (knowledgeGrid2.start.x, knowledgeGrid2.start.y)
            for index, current in enumerate(path):
                gridObj2.iterations += 1
                alreadyConfirmed = False
                if knowledgeGrid2.cellMap.get((current.x, current.y)) is None:
                    knowledgeGrid2.cellMap[(current.x, current.y)] = current

                else:
                    alreadyConfirmed = True
                    current = knowledgeGrid2.cellMap[(current.x, current.y)]

                current.parent = prev

                if current.x == goal.x and current.y == goal.y:
                    return knowledgeGrid2.path
                #Ucode Start
                #Project2MovementDict['Grid'].append(knowledgeGrid.world.flatten().tolist())
                
                #Ucode End
                # hit blocked cell
                if grid[current.x, current.y] == 1:
                    current.confirmed = True
                    # print("Blocked by ", current.x, current.y)
                    knowledgeGrid2.world[current.x, current.y] = 1

                    KnowledgeGridMinusInferance.world[current.x, current.y] = 1

                    # setting restart point
                    knowledgeGrid2.start = gridObj2.cellMap[current.parent[0], current.parent[1]]

                    KnowledgeGridMinusInferance.world[current.parent[0], current.parent[1]]= 2

                    # adding to the cell map
                    knowledgeGrid2.cellMap[(current.x, current.y)] = current

                    # inform the neighbours that this is blocked
                    if alreadyConfirmed == False:
                        informNeighboursAboutNewBlockedCell(current)

                    solver.solve_constraint(current, 1)

                    return None

                else:
                    # can only sense neighbours of free cells
                    visited_grid[current.x, current.y] += 1
                    current.confirmed = True
                    current.visited = True
                    current.total_neighbours = gridObj2.cellMap[(current.x, current.y)].total_neighbours
                    current.total_blocked_neighbours = gridObj2.cellMap[(current.x, current.y)].total_blocked_neighbours
                    InferanceGrid.world[(current.x, current.y)] = gridObj2.cellMap[(current.x, current.y)].total_blocked_neighbours
                    if alreadyConfirmed == False:
                        current = getKnownNeighbours(current)

                    knowledgeGrid2.cellMap[(current.x, current.y)] = current
                    knowledgeGrid2.path.append(current)
                    inferenceEngine(current)

                    # inform the neighbours that this is free
                    if alreadyConfirmed == False:
                        informNeigboursAboutNewFreeCell(current)

                    # advanced inference
                    agent_four_inference(current, current.total_blocked_neighbours)
                    solver.solve_constraint(current, 0)
                    prev = (current.x, current.y)


        def countBlockedNeighboursInRealGrid():
            # populate the information about blocked neighbours in the real grid
            for i in range(n):
                for j in range(n):
                    cell = Cell(i, j, 0, 0, (0, 0))
                    cell = senseNeighbours(cell)

                    # create object for all cells in the grid
                    gridObj2.cellMap[(cell.x, cell.y)] = cell

        def repeatedAStar():

            #printGridInItsFullGlory(visited_grid)
            countBlockedNeighboursInRealGrid()

            # mark start as confirmed
            knowledgeGrid2.start.confirmed = True
            knowledgeGrid2.start.total_blocked_neighbours = gridObj2.cellMap[(0, 0)].total_blocked_neighbours
            knowledgeGrid2.cellMap[(0, 0)] = knowledgeGrid2.start

            agent_four_inference(knowledgeGrid2.start, knowledgeGrid2.start.total_blocked_neighbours)

            while True:
                if len(knowledgeGrid2.path) > 0:
                    previousStartPoint = knowledgeGrid2.path[len(knowledgeGrid2.path) - 1]

                    # if we are restarting from the same point where we restarted previously, then dont add this to the path again
                    if knowledgeGrid2.start.x != previousStartPoint.x and knowledgeGrid2.start.y != previousStartPoint.y:
                        knowledgeGrid2.path.append(knowledgeGrid2.start)

                path = AStar(knowledgeGrid2, knowledgeGrid2.start)
                if path is None:
                    continue

                elif path == False:
                    return False

                else:
                    leng = 2
                    knowledgeGrid2.world[0, 0] = 5
                    knowledgeGrid2.world[knowledgeGrid2.start.x, knowledgeGrid2.start.y] = 5

                    for i in knowledgeGrid2.path:
                        leng +=1
                        knowledgeGrid2.world[i.x, i.y] = 5

                    knowledgeGrid2.world[knowledgeGrid2.goal.x, knowledgeGrid2.goal.y] = 5
                    print(gridObj2.world)
                    print(knowledgeGrid2.world)
                    print("Total iterations = ", gridObj2.iterations) 
                    return (True, gridObj2.iterations, leng)
        return repeatedAStar()
################## Neural Agent on Agent 1 #########################    
    def NeuralVoyage(n,p):

      def Direct(currentGrid, current):
          world = currentGrid
          world2 = world
          world[current.x,current.y]=2
          world[knowledgeGrid3.goal.x,knowledgeGrid3.goal.y]=3
          #print(world)
          world = world.flatten()
          test_in = np.reshape(world, (-1, 101, 101) )
          predicted_move = model.predict(test_in)
          #print(predicted_move)
          predicted_move = predicted_move[0].tolist()
          direction = predicted_move.index(np.max(predicted_move))
          fourth = predicted_move.index(np.min(predicted_move))
          predicted_move[direction] = 0
          second = predicted_move.index(np.max(predicted_move))
          predicted_move[second] = 0
          third = predicted_move.index(np.max(predicted_move))
          predicted_move[third] = 0
          world = world2
          world[current.x,current.y]=0
          #print(direction, second, third, fourth)
          return (direction, second, third, fourth)
          
      # this is the planning phase  
      def NN(currentGrid, start):
        gridObj3.allCells = {}
        gridObj3.allCells[(start.x, start.y)] = start
        #print('Starting at: ', (start.x, start.y))
        current = start
        prev = (current.x, current.y)
        goal = gridObj3.goal
        while (current.x, current.y) != (goal.x, goal.y):
          #print((current.x, current.y))
          rightCell = Cell(current.x + 1, current.y, current.cost_from_start + 1, 0,
                          (current.x, current.y))  # right neighbour
          downCell = Cell(current.x, current.y + 1, current.cost_from_start + 1, 0,
                          (current.x, current.y))  # down neighbour
          leftCell = Cell(current.x - 1, current.y, current.cost_from_start + 1, 0,
                          (current.x, current.y))  # left neighbour
          topCell = Cell(current.x, current.y - 1, current.cost_from_start + 1, 0,
                          (current.x, current.y))  # top neighbour

          neighbors = [rightCell, leftCell, topCell, downCell]  # calculate the f(n) for each neighbour of current
          move = Direct(knowledgeGrid3.world, current)
          get1 = gridObj3.allCells.get((neighbors[move[0]].x, neighbors[move[0]].y))
          get2 = gridObj3.allCells.get((neighbors[move[1]].x, neighbors[move[1]].y))
          get3 = gridObj3.allCells.get((neighbors[move[2]].x, neighbors[move[2]].y))
          get4 = gridObj3.allCells.get((neighbors[move[3]].x, neighbors[move[3]].y))
          if isCellOpen(neighbors[move[0]], n, currentGrid) and get1 is None:
            gridObj3.allCells[(neighbors[move[0]].x, neighbors[move[0]].y)] = neighbors[move[0]]
            current = neighbors[move[0]]
          elif isCellOpen(neighbors[move[1]], n, currentGrid) and get2 is None:
            gridObj3.allCells[(neighbors[move[1]].x, neighbors[move[1]].y)] = neighbors[move[1]]
            current = neighbors[move[1]]
          elif isCellOpen(neighbors[move[2]], n, currentGrid) and get3 is None:
            gridObj3.allCells[(neighbors[move[2]].x, neighbors[move[2]].y)] = neighbors[move[2]]
            current = neighbors[move[2]]
          elif isCellOpen(neighbors[move[3]], n, currentGrid) and get4 is None: 
            gridObj3.allCells[(neighbors[move[3]].x, neighbors[move[3]].y)] = neighbors[move[3]]
            current = neighbors[move[3]]
          else:
            if (current.x, current.y) == prev:
              return False
            prev = (current.x, current.y)
            pass
        path = reconstructPath(current, start)
        return executePlannedPath(path)

      def reconstructPath(current, start):
        #print('Reconstructing')
        path = collections.deque() 
        while True:
          if (current.x, current.y) == (start.x, start.y):
            break
          path.appendleft(current)
          current = gridObj3.allCells[(current.parent[0], current.parent[1])]
        return path

      def executePlannedPath(path):
        #print('Executing')
        grid = gridObj3.world
        goal = gridObj3.goal
        isPathFeasible = True

        for index, current in enumerate(path):
          #print("Visiting ", current)
          gridObj3.iterations += 1
          #print(current.x, current.y)

          #print(current.x-current.parent[0], current.y-current.parent[1])
          if current.x == goal.x and current.y == goal.y:
            return gridObj3.path

          if grid[current.x][current.y] == 1:
            knowledgeGrid3.world[current.x][current.y] = 1
            isPathFeasible = False

            # this should be the restart point now
            setRestartPoint(gridObj3.allCells[current.parent[0], current.parent[1]], 0)
            return None

          else:
            gridObj3.path.append(current)

          # update the neighbours in the actual grid   
          # update the blockedNeighboursCount   
          if isCellWithinGrid(current.x + 1, current.y, n) and grid[current.x + 1][current.y] == 1:
            knowledgeGrid3[current.x + 1, current.y] = 1
          
          if isCellWithinGrid(current.x + 1, current.y, n) and grid[current.x + 1][current.y] == 1:
            knowledgeGrid3[current.x][current.y + 1] = 1
          
          if isCellWithinGrid(current.x - 1, current.y, n) and grid[current.x - 1][current.y] == 1:
            knowledgeGrid3[current.x + 1][current.y] = 1
          
          if isCellWithinGrid(current.x - 1, current.y, n) and grid[current.x][current.y - 1] == 1:
            knowledgeGrid3[current.x + 1][current.y] = 1
      
      def setRestartPoint(restartPoint, restartValue):
        for i in range(0, restartValue, 1):
          restartPoint = gridObj3.allCells[(restartPoint.parent[0], restartPoint.parent[1])]

        gridObj3.start = restartPoint
        return

      def printGridInItsFullGlory(n, world):
        for i in range(0, n, 1):
          for j in range(0, n, 1):
            print(world[i, j], end=' ')
          print()

      def repeatedNN():
        #printGridInItsFullGlory(n, gridObj.world)
        while True:
            if len(gridObj3.path) > 0:
              previousStartPoint = gridObj3.path[len(gridObj3.path)-1]

              # if we are restarting from the same point where we restarted previously, then dont add this to the path again
              if gridObj3.start.x != previousStartPoint.x and gridObj3.start.y != previousStartPoint.y:
                gridObj3.path.append(gridObj3.start)

            path = NN(knowledgeGrid3.world, gridObj3.start)
            if path is None:
              continue

            elif path == False:
              return False

            # final path to goal found
            else:
              leng =2
              gridObj3.world[0][0] = 5
              gridObj3.world[gridObj3.start.x][gridObj3.start.y] = 5
              
              for i in gridObj3.path:
                leng += 1
                gridObj3.world[i.x][i.y] = 5

              gridObj3.world[gridObj3.goal.x][gridObj3.goal.y] = 5
              print(gridObj3.world)
              #printGridInItsFullGlory()
              print("Total iterations = ", gridObj3.iterations)
              return (True, gridObj3.iterations, leng)

      return repeatedNN()

    start = timer()
    NA = NeuralVoyage(n, p)
    end = timer()
    if NA == False:
      Comparison(n,p)
    print(end-start)
    start = timer()
    A1 = Voyage(n, p)
    end = timer()
    print(end-start)
    start = timer()
    A2 = PartialSensing(n, p)
    end = timer()
    print(end-start)
Comparison(101, 0.3)